# POC

In [11]:
"""
╔══════════════════════════════════════════════════════════════════════════════╗
║  SWARM-MAMBA: Asynchronous Parallel State Machines with Emergent Consensus   ║
║  Proof of Concept for Q1 Research Validation                                 ║
║                                                                              ║
║  Decision Thresholds:                                                        ║
║  • Accuracy Delta: Swarm within 10% of baseline                              ║
║  • Speedup: ≥1.0x with k=4 agents                                            ║
║  • Consensus Rate: ≥60%                                                      ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION & CONSTANTS
# ============================================================================

@dataclass
class ExperimentConfig:
    """Central configuration for reproducibility"""
    seed: int = 42
    img_size: int = 64
    patch_size: int = 8
    n_classes: int = 2
    n_train: int = 200
    n_test: int = 56
    epochs: int = 12
    batch_size: int = 32
    lr: float = 1e-3
    
    # Decision thresholds
    accuracy_threshold: float = 0.65
    relative_accuracy_threshold: float = 0.90
    speedup_threshold: float = 1.0
    consensus_threshold: float = 0.60

CONFIG = ExperimentConfig()

# Set seeds
torch.manual_seed(CONFIG.seed)
np.random.seed(CONFIG.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(CONFIG.seed)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ============================================================================
# SECTION 1: SIMPLIFIED SELECTIVE STATE SPACE MODEL (SSM)
# ============================================================================

class SelectiveSSM(nn.Module):
    """
    Simplified Selective State Space Model capturing Mamba's core innovation:
    Input-dependent selection mechanism for dynamic context modeling.
    
    Key insight: The selection mechanism (Δ, B, C) is computed from input,
    enabling the model to selectively propagate or forget information.
    
    Reference: Gu & Dao (2024), "Mamba: Linear-Time Sequence Modeling"
    """
    def __init__(
        self, 
        d_model: int, 
        d_state: int = 16, 
        expand_factor: int = 2,
        dt_rank: str = "auto"
    ):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_inner = d_model * expand_factor
        self.dt_rank = d_state if dt_rank == "auto" else dt_rank
        
        # Input projection: x → (x_branch, gate)
        self.in_proj = nn.Linear(d_model, self.d_inner * 2, bias=False)
        
        # Depthwise conv for local context
        self.conv1d = nn.Conv1d(
            self.d_inner, self.d_inner,
            kernel_size=4, padding=2,
            groups=self.d_inner
        )
        
        # Selection mechanism projections
        # Projects to (dt, B, C) - all input-dependent
        self.x_proj = nn.Linear(self.d_inner, self.dt_rank + d_state * 2, bias=False)
        self.dt_proj = nn.Linear(self.dt_rank, self.d_inner, bias=True)
        
        # Initialize dt bias for stability
        dt_init_std = self.dt_rank ** -0.5
        nn.init.uniform_(self.dt_proj.bias, -dt_init_std, dt_init_std)
        
        # Learnable state matrix A (discretized log-space)
        A = torch.arange(1, d_state + 1, dtype=torch.float32)
        self.A_log = nn.Parameter(torch.log(A))
        
        # Output normalization and projection
        self.norm = nn.LayerNorm(self.d_inner)
        self.out_proj = nn.Linear(self.d_inner, d_model, bias=False)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (batch, seq_len, d_model)
        Returns:
            (batch, seq_len, d_model)
        """
        batch, seq_len, _ = x.shape
        
        # Step 1: Input projection and split into branches
        xz = self.in_proj(x)  # (B, L, 2*d_inner)
        x_branch, z = xz.chunk(2, dim=-1)  # Each: (B, L, d_inner)
        
        # Step 2: Local convolution for short-range dependencies
        x_branch = x_branch.transpose(1, 2)  # (B, d_inner, L)
        x_branch = self.conv1d(x_branch)[:, :, :seq_len]  # Truncate padding
        x_branch = x_branch.transpose(1, 2)  # (B, L, d_inner)
        x_branch = F.silu(x_branch)
        
        # Step 3: Compute input-dependent selection parameters
        x_dbl = self.x_proj(x_branch)  # (B, L, dt_rank + 2*d_state)
        dt, B, C = torch.split(
            x_dbl, 
            [self.dt_rank, self.d_state, self.d_state], 
            dim=-1
        )
        
        # Step 4: Discretize dt (step size)
        dt = F.softplus(self.dt_proj(dt))  # (B, L, d_inner)
        
        # Step 5: Compute discretized A
        A = -torch.exp(self.A_log)  # (d_state,)
        
        # Step 6: Selective scan (simplified parallel approximation for PoC)
        # Full implementation would use associative scan
        y = self._selective_scan_simple(x_branch, dt, A, B, C)
        
        # Step 7: Output gating and projection
        y = self.norm(y)
        y = y * F.silu(z)  # Gating mechanism
        return self.out_proj(y)
    
    def _selective_scan_simple(
        self, 
        x: torch.Tensor,      # (B, L, d_inner)
        dt: torch.Tensor,     # (B, L, d_inner)
        A: torch.Tensor,      # (d_state,)
        B: torch.Tensor,      # (B, L, d_state)
        C: torch.Tensor       # (B, L, d_state)
    ) -> torch.Tensor:
        """
        Simplified selective scan for PoC.
        Uses chunk-wise processing for memory efficiency.
        """
        batch, seq_len, d_inner = x.shape
        d_state = self.d_state
        
        # Process in chunks for efficiency
        chunk_size = min(32, seq_len)
        n_chunks = (seq_len + chunk_size - 1) // chunk_size
        
        outputs = []
        h = torch.zeros(batch, d_inner, d_state, device=x.device)
        
        for c in range(n_chunks):
            start = c * chunk_size
            end = min(start + chunk_size, seq_len)
            
            for t in range(start, end):
                # Discretized state update
                dt_t = dt[:, t, :].unsqueeze(-1)  # (B, d_inner, 1)
                dA = torch.exp(A.view(1, 1, -1) * dt_t)  # (B, d_inner, d_state)
                dB = dt_t * B[:, t, :].unsqueeze(1)  # (B, d_inner, d_state)
                
                # State update: h = dA * h + dB * x
                h = dA * h + dB * x[:, t, :].unsqueeze(-1)
                
                # Output: y = C @ h
                y_t = (h * C[:, t, :].unsqueeze(1)).sum(dim=-1)  # (B, d_inner)
                outputs.append(y_t)
        
        return torch.stack(outputs, dim=1)  # (B, L, d_inner)


# ============================================================================
# SECTION 2: MICRO-MAMBA AGENT
# ============================================================================

class MicroMamba(nn.Module):
    """
    Lightweight Mamba agent for swarm processing.
    Each agent processes a spatial partition of the image.
    
    Design principle: Keep individual agents small (O(n/k) parameters)
    while maintaining expressive power through communication.
    """
    def __init__(
        self, 
        d_model: int, 
        n_layers: int = 2, 
        d_state: int = 16,
        dropout: float = 0.1
    ):
        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        
        # Stack of SSM blocks with residual connections
        self.blocks = nn.ModuleList()
        for _ in range(n_layers):
            self.blocks.append(nn.ModuleDict({
                'norm': nn.LayerNorm(d_model),
                'ssm': SelectiveSSM(d_model, d_state),
                'dropout': nn.Dropout(dropout)
            }))
        
        # Agent-specific learned embedding
        self.agent_token = nn.Parameter(torch.randn(1, 1, d_model) * 0.02)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Process local partition.
        
        Args:
            x: (batch, seq_len, d_model) - partition tokens
        Returns:
            (batch, seq_len, d_model) - processed tokens
        """
        batch = x.shape[0]
        
        # Prepend agent token for identity-aware processing
        agent_tok = self.agent_token.expand(batch, -1, -1)
        x = torch.cat([agent_tok, x], dim=1)
        
        # Process through SSM blocks
        for block in self.blocks:
            residual = x
            x = block['norm'](x)
            x = block['ssm'](x)
            x = block['dropout'](x)
            x = residual + x
        
        # Remove agent token
        return x[:, 1:, :]
    
    def get_summary_state(self, x: torch.Tensor) -> torch.Tensor:
        """Get compressed state for inter-agent communication"""
        return x.mean(dim=1)  # (batch, d_model)


# ============================================================================
# SECTION 3: INTER-AGENT COMMUNICATION
# ============================================================================

class DifferentiableMessagePassing(nn.Module):
    """
    Learned message passing between swarm agents.
    
    Key innovation: Agents broadcast compressed states and aggregate
    messages through learned attention, enabling emergent coordination.
    
    This mimics cortical column communication in biological vision.
    """
    def __init__(
        self, 
        d_model: int, 
        n_agents: int,
        compression_ratio: int = 2,
        n_heads: int = 4
    ):
        super().__init__()
        self.n_agents = n_agents
        self.d_model = d_model
        self.d_compressed = d_model // compression_ratio
        
        # State compression/decompression
        self.compress = nn.Sequential(
            nn.Linear(d_model, self.d_compressed),
            nn.GELU(),
            nn.LayerNorm(self.d_compressed)
        )
        self.decompress = nn.Linear(self.d_compressed, d_model)
        
        # Multi-head cross-agent attention
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)
        
        # Learnable mixing coefficient
        self.alpha = nn.Parameter(torch.tensor(0.3))
        
        # Topology mask (optional: can implement local connectivity)
        self.register_buffer(
            'topology_mask',
            torch.ones(n_agents, n_agents) - torch.eye(n_agents)
        )
        
    def forward(self, agent_states: List[torch.Tensor]) -> List[torch.Tensor]:
        """
        Exchange messages between agents.
        
        Args:
            agent_states: List of (batch, seq_len, d_model) tensors
        Returns:
            Updated agent states after message exchange
        """
        batch = agent_states[0].shape[0]
        
        # Step 1: Compress each agent's state for communication
        summaries = [self.compress(s.mean(dim=1)) for s in agent_states]
        stacked_summaries = torch.stack(summaries, dim=1)  # (B, n_agents, d_compressed)
        
        # Step 2: Decompress for attention computation
        decompressed = self.decompress(stacked_summaries)  # (B, n_agents, d_model)
        
        # Step 3: Multi-head attention across agents
        Q = self.q_proj(decompressed)  # (B, n_agents, d_model)
        K = self.k_proj(decompressed)
        V = self.v_proj(decompressed)
        
        # Reshape for multi-head
        Q = Q.view(batch, self.n_agents, self.n_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch, self.n_agents, self.n_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch, self.n_agents, self.n_heads, self.head_dim).transpose(1, 2)
        
        # Scaled dot-product attention
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.head_dim)
        
        # Apply topology mask (agents don't attend to themselves)
        mask = self.topology_mask.unsqueeze(0).unsqueeze(0)  # (1, 1, n_agents, n_agents)
        scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_output = torch.matmul(attn_weights, V)  # (B, n_heads, n_agents, head_dim)
        
        # Reshape back
        attn_output = attn_output.transpose(1, 2).reshape(batch, self.n_agents, self.d_model)
        messages = self.out_proj(attn_output)  # (B, n_agents, d_model)
        
        # Step 4: Update agent states with received messages
        alpha = torch.sigmoid(self.alpha)
        updated_states = []
        
        for i, state in enumerate(agent_states):
            # Broadcast message to all tokens in the partition
            msg = messages[:, i:i+1, :].expand(-1, state.shape[1], -1)
            updated = state + alpha * msg
            updated_states.append(updated)
        
        return updated_states


# ============================================================================
# SECTION 4: CONSENSUS MODULE
# ============================================================================

class EmergentConsensus(nn.Module):
    """
    Aggregates agent predictions into unified output.
    
    Two consensus mechanisms:
    1. Learned weighted voting (soft attention over agents)
    2. Contrastive alignment (agents trained to agree)
    
    Also tracks individual agent predictions for interpretability.
    """
    def __init__(self, d_model: int, n_agents: int, n_classes: int):
        super().__init__()
        self.n_agents = n_agents
        self.d_model = d_model
        self.n_classes = n_classes
        
        # Learnable agent importance weights
        self.agent_importance = nn.Parameter(torch.ones(n_agents) / n_agents)
        
        # Per-agent classification heads (for diversity and interpretability)
        self.agent_heads = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm(d_model),
                nn.Linear(d_model, d_model // 2),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(d_model // 2, n_classes)
            )
            for _ in range(n_agents)
        ])
        
        # Unified consensus head (processes concatenated representations)
        self.consensus_head = nn.Sequential(
            nn.LayerNorm(d_model * n_agents),
            nn.Linear(d_model * n_agents, d_model),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(d_model, n_classes)
        )
        
    def forward(
        self, 
        agent_states: List[torch.Tensor]
    ) -> Tuple[torch.Tensor, List[torch.Tensor], float]:
        """
        Compute consensus prediction.
        
        Returns:
            consensus_logits: (batch, n_classes)
            individual_logits: List of (batch, n_classes)
            consensus_rate: Scalar measuring agent agreement
        """
        # Pool each agent's state to single vector
        pooled_states = [s.mean(dim=1) for s in agent_states]  # List of (B, d_model)
        
        # Individual agent predictions
        individual_logits = [
            head(state) for head, state in zip(self.agent_heads, pooled_states)
        ]
        
        # Method 1: Weighted average of agent states
        weights = F.softmax(self.agent_importance, dim=0)
        weighted_state = sum(w * s for w, s in zip(weights, pooled_states))
        
        # Method 2: Concatenated consensus
        concat_states = torch.cat(pooled_states, dim=-1)  # (B, d_model * n_agents)
        consensus_logits = self.consensus_head(concat_states)
        
        # Compute consensus rate (how much agents agree)
        with torch.no_grad():
            individual_preds = torch.stack([
                torch.argmax(logits, dim=-1) for logits in individual_logits
            ], dim=1)  # (B, n_agents)
            
            # Mode prediction
            mode_pred = torch.mode(individual_preds, dim=1).values  # (B,)
            agreement = (individual_preds == mode_pred.unsqueeze(1)).float().mean()
        
        return consensus_logits, individual_logits, agreement.item()


# ============================================================================
# SECTION 5: SWARM-MAMBA MAIN ARCHITECTURE
# ============================================================================

class SwarmMamba(nn.Module):
    """
    Swarm-Mamba: Asynchronous Parallel State Machines with Emergent Consensus
    
    Architecture:
    1. Patch embedding (shared across agents)
    2. Spatial partitioning among k agents
    3. Parallel processing by micro-Mambas
    4. Iterative message passing rounds
    5. Consensus aggregation
    
    Key benefits:
    - True O(n/k) parallel time complexity per agent
    - Graceful degradation (built-in redundancy)
    - Interpretable agent specialization
    """
    def __init__(
        self,
        img_size: int = 64,
        patch_size: int = 8,
        in_channels: int = 3,
        n_agents: int = 4,
        d_model: int = 64,
        n_layers: int = 2,
        n_classes: int = 2,
        n_comm_rounds: int = 2,
        d_state: int = 16
    ):
        super().__init__()
        self.n_agents = n_agents
        self.n_comm_rounds = n_comm_rounds
        self.d_model = d_model
        self.img_size = img_size
        self.patch_size = patch_size
        
        # Calculate patch grid
        self.n_patches_side = img_size // patch_size
        self.n_patches = self.n_patches_side ** 2
        
        # Patch embedding (Conv2D based)
        self.patch_embed = nn.Conv2d(
            in_channels, d_model,
            kernel_size=patch_size,
            stride=patch_size
        )
        
        # Learnable position embeddings
        self.pos_embed = nn.Parameter(
            torch.randn(1, self.n_patches, d_model) * 0.02
        )
        
        # Swarm of micro-Mamba agents
        self.agents = nn.ModuleList([
            MicroMamba(d_model, n_layers, d_state)
            for _ in range(n_agents)
        ])
        
        # Inter-agent communication
        self.message_passing = DifferentiableMessagePassing(d_model, n_agents)
        
        # Consensus module
        self.consensus = EmergentConsensus(d_model, n_agents, n_classes)
        
    def partition_spatially(self, x: torch.Tensor) -> List[torch.Tensor]:
        """
        Partition patch sequence among agents using spatial regions.
        
        For 2D grid partitioning (more biologically plausible than random):
        - Divide image into k spatial quadrants/regions
        - Each agent processes patches from its region
        """
        batch, n_patches, d = x.shape
        
        # Calculate patches per agent
        patches_per_agent = n_patches // self.n_agents
        
        partitions = []
        for i in range(self.n_agents):
            start_idx = i * patches_per_agent
            end_idx = start_idx + patches_per_agent if i < self.n_agents - 1 else n_patches
            partitions.append(x[:, start_idx:end_idx, :].clone())
        
        return partitions
    
    def forward(
        self, 
        x: torch.Tensor
    ) -> Tuple[torch.Tensor, List[torch.Tensor], float]:
        """
        Forward pass with parallel agent processing.
        
        Args:
            x: (batch, channels, height, width)
            
        Returns:
            logits: (batch, n_classes) - consensus prediction
            individual_logits: List of agent predictions
            consensus_rate: Agreement metric
        """
        batch = x.shape[0]
        
        # Step 1: Patch embedding
        x = self.patch_embed(x)  # (B, d_model, H', W')
        x = x.flatten(2).transpose(1, 2)  # (B, n_patches, d_model)
        x = x + self.pos_embed
        
        # Step 2: Partition among agents
        partitions = self.partition_spatially(x)
        
        # Step 3: Initial parallel processing
        # Note: In production, this would use torch.multiprocessing or GPU parallelism
        agent_states = []
        for agent, partition in zip(self.agents, partitions):
            state = agent(partition)
            agent_states.append(state)
        
        # Step 4: Communication rounds
        for round_idx in range(self.n_comm_rounds):
            # Message exchange
            agent_states = self.message_passing(agent_states)
            
            # Additional local processing after receiving messages
            agent_states = [
                agent(state) 
                for agent, state in zip(self.agents, agent_states)
            ]
        
        # Step 5: Consensus
        consensus_logits, individual_logits, consensus_rate = self.consensus(agent_states)
        
        return consensus_logits, individual_logits, consensus_rate


# ============================================================================
# SECTION 6: BASELINE SINGLE MAMBA (FOR COMPARISON)
# ============================================================================

class SingleMamba(nn.Module):
    """
    Baseline: Single monolithic Mamba model.
    Parameters scaled to match total Swarm-Mamba params for fair comparison.
    """
    def __init__(
        self,
        img_size: int = 64,
        patch_size: int = 8,
        in_channels: int = 3,
        d_model: int = 128,
        n_layers: int = 4,
        n_classes: int = 2,
        d_state: int = 16
    ):
        super().__init__()
        
        self.n_patches = (img_size // patch_size) ** 2
        
        # Patch embedding
        self.patch_embed = nn.Conv2d(
            in_channels, d_model,
            kernel_size=patch_size,
            stride=patch_size
        )
        
        # Position embedding
        self.pos_embed = nn.Parameter(
            torch.randn(1, self.n_patches, d_model) * 0.02
        )
        
        # SSM blocks
        self.blocks = nn.ModuleList()
        for _ in range(n_layers):
            self.blocks.append(nn.ModuleDict({
                'norm': nn.LayerNorm(d_model),
                'ssm': SelectiveSSM(d_model, d_state),
            }))
        
        # Classification head
        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(d_model // 2, n_classes)
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Patch embedding
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2)
        x = x + self.pos_embed
        
        # Process through SSM blocks
        for block in self.blocks:
            residual = x
            x = block['norm'](x)
            x = block['ssm'](x)
            x = residual + x
        
        # Global average pooling + classification
        x = x.mean(dim=1)
        return self.head(x)


# ============================================================================
# SECTION 7: SYNTHETIC DATA GENERATION
# ============================================================================

def generate_medical_imaging_dataset(
    n_samples: int,
    img_size: int = 64,
    n_classes: int = 2
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Generate synthetic dataset mimicking histopathology patterns.
    
    Class 0 (Benign): Uniform cellular texture
    Class 1 (Malignant): Irregular patterns with focal lesions
    
    This is for PoC only - real experiments use BreaKHis dataset.
    """
    images = []
    labels = []
    
    for i in range(n_samples):
        label = i % n_classes
        
        # Base tissue texture
        img = torch.randn(3, img_size, img_size) * 0.15
        
        # Add class-specific patterns
        if label == 0:  # Benign
            # Uniform coloring with slight variations
            base_color = torch.randn(3, 1, 1) * 0.3 + 0.5
            img = img + base_color
            
            # Regular cellular pattern
            for cx in range(8, img_size - 8, 12):
                for cy in range(8, img_size - 8, 12):
                    y, x = torch.meshgrid(
                        torch.arange(img_size), 
                        torch.arange(img_size),
                        indexing='ij'
                    )
                    dist = ((x - cx)**2 + (y - cy)**2).float()
                    mask = dist < 16
                    img[0, mask] += 0.1
                    
        else:  # Malignant
            # Irregular base coloring
            img = img + 0.4
            
            # Add irregular lesion spots (key diagnostic feature)
            n_lesions = np.random.randint(3, 8)
            for _ in range(n_lesions):
                cx = np.random.randint(10, img_size - 10)
                cy = np.random.randint(10, img_size - 10)
                
                # Irregular shape (ellipse + noise)
                rx = np.random.randint(4, 12)
                ry = np.random.randint(4, 12)
                
                y, x = torch.meshgrid(
                    torch.arange(img_size),
                    torch.arange(img_size),
                    indexing='ij'
                )
                dist = ((x - cx)**2 / rx**2 + (y - cy)**2 / ry**2).float()
                mask = dist < 1
                
                # Darker lesion center
                intensity = torch.randn(1) * 0.2 - 0.3
                for c in range(3):
                    img[c, mask] += intensity + torch.randn(1) * 0.1
        
        # Normalize to [0, 1] range
        img = torch.clamp(img, 0, 1)
        
        images.append(img)
        labels.append(label)
    
    return torch.stack(images), torch.tensor(labels, dtype=torch.long)


# ============================================================================
# SECTION 8: TRAINING & EVALUATION
# ============================================================================

def train_model(
    model: nn.Module,
    train_data: Tuple[torch.Tensor, torch.Tensor],
    epochs: int,
    is_swarm: bool = False,
    verbose: bool = True
) -> nn.Module:
    """Train model with standard CE loss + optional consistency regularization."""
    
    X_train, y_train = train_data
    X_train, y_train = X_train.to(DEVICE), y_train.to(DEVICE)
    model = model.to(DEVICE)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG.lr, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        n_batches = 0
        
        # Shuffle data
        perm = torch.randperm(len(X_train))
        X_train, y_train = X_train[perm], y_train[perm]
        
        for i in range(0, len(X_train), CONFIG.batch_size):
            batch_x = X_train[i:i + CONFIG.batch_size]
            batch_y = y_train[i:i + CONFIG.batch_size]
            
            optimizer.zero_grad()
            
            if is_swarm:
                logits, individual_logits, _ = model(batch_x)
                
                # Primary loss
                loss = criterion(logits, batch_y)
                
                # Consistency loss: all agents should predict same class
                for ind_logits in individual_logits:
                    loss += 0.1 * criterion(ind_logits, batch_y)
                    
                # Diversity regularization: prevent agent collapse
                # (agents should use different features)
                agent_features = [model.agents[j].agent_token for j in range(len(model.agents))]
                if len(agent_features) > 1:
                    for j in range(len(agent_features) - 1):
                        sim = F.cosine_similarity(
                            agent_features[j].flatten(),
                            agent_features[j+1].flatten(),
                            dim=0
                        )
                        loss += 0.01 * sim  # Minimize similarity
            else:
                logits = model(batch_x)
                loss = criterion(logits, batch_y)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            n_batches += 1
        
        scheduler.step()
        
        if verbose and (epoch + 1) % 4 == 0:
            avg_loss = total_loss / n_batches
            print(f"    Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    
    return model


def evaluate_model(
    model: nn.Module,
    test_data: Tuple[torch.Tensor, torch.Tensor],
    is_swarm: bool = False
) -> Dict[str, float]:
    """Evaluate model on test set."""
    
    X_test, y_test = test_data
    X_test, y_test = X_test.to(DEVICE), y_test.to(DEVICE)
    model = model.to(DEVICE)
    
    model.eval()
    correct = 0
    total_time = 0
    consensus_rates = []
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for i in range(len(X_test)):
            x = X_test[i:i+1]
            
            # Time the inference
            if DEVICE.type == 'cuda':
                torch.cuda.synchronize()
            start = time.perf_counter()
            
            if is_swarm:
                logits, _, cons_rate = model(x)
                consensus_rates.append(cons_rate)
            else:
                logits = model(x)
            
            if DEVICE.type == 'cuda':
                torch.cuda.synchronize()
            end = time.perf_counter()
            
            total_time += (end - start) * 1000  # Convert to ms
            
            pred = torch.argmax(logits, dim=-1)
            correct += (pred == y_test[i]).sum().item()
            
            all_preds.append(pred.cpu().item())
            all_labels.append(y_test[i].cpu().item())
    
    accuracy = correct / len(X_test)
    avg_time_ms = total_time / len(X_test)
    avg_consensus = np.mean(consensus_rates) if consensus_rates else 1.0
    
    # Per-class accuracy
    class_acc = {}
    for c in range(CONFIG.n_classes):
        mask = [l == c for l in all_labels]
        if sum(mask) > 0:
            class_correct = sum(p == l for p, l, m in zip(all_preds, all_labels, mask) if m)
            class_acc[f'class_{c}_acc'] = class_correct / sum(mask)
    
    return {
        'accuracy': accuracy,
        'inference_time_ms': avg_time_ms,
        'consensus_rate': avg_consensus,
        **class_acc
    }


# ============================================================================
# SECTION 9: TEST SUITE
# ============================================================================

def run_test_suite() -> bool:
    """Comprehensive test suite for all components."""
    
    print("\n" + "=" * 70)
    print(" RUNNING TEST SUITE")
    print("=" * 70)
    
    all_passed = True
    
    # Test 1: SelectiveSSM
    print("\n[TEST 1] SelectiveSSM forward pass...")
    try:
        ssm = SelectiveSSM(d_model=32, d_state=8).to(DEVICE)
        x = torch.randn(2, 16, 32).to(DEVICE)
        out = ssm(x)
        assert out.shape == x.shape, f"Shape mismatch: {out.shape} vs {x.shape}"
        assert not torch.isnan(out).any(), "NaN in output"
        print("  ✓ Output shape and values correct")
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        all_passed = False
    
    # Test 2: MicroMamba
    print("\n[TEST 2] MicroMamba agent processing...")
    try:
        agent = MicroMamba(d_model=32, n_layers=2).to(DEVICE)
        x = torch.randn(2, 8, 32).to(DEVICE)
        out = agent(x)
        assert out.shape == x.shape, f"Shape mismatch"
        print("  ✓ Agent processes partition correctly")
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        all_passed = False
    
    # Test 3: Message Passing
    print("\n[TEST 3] Inter-agent message passing...")
    try:
        mp = DifferentiableMessagePassing(d_model=32, n_agents=4).to(DEVICE)
        states = [torch.randn(2, 8, 32).to(DEVICE) for _ in range(4)]
        updated = mp(states)
        assert len(updated) == 4, "Wrong number of states returned"
        assert all(u.shape == s.shape for u, s in zip(updated, states))
        # Check that states actually changed (communication happened)
        changed = any(not torch.allclose(u, s) for u, s in zip(updated, states))
        assert changed, "States unchanged after message passing"
        print("  ✓ Message passing modifies states correctly")
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        all_passed = False
    
    # Test 4: Consensus Module
    print("\n[TEST 4] Emergent consensus...")
    try:
        cons = EmergentConsensus(d_model=32, n_agents=4, n_classes=2).to(DEVICE)
        states = [torch.randn(2, 8, 32).to(DEVICE) for _ in range(4)]
        logits, ind_logits, rate = cons(states)
        assert logits.shape == (2, 2), f"Wrong consensus shape: {logits.shape}"
        assert len(ind_logits) == 4, "Wrong number of individual predictions"
        assert 0 <= rate <= 1, f"Invalid consensus rate: {rate}"
        print(f"  ✓ Consensus produces valid output (rate: {rate:.2f})")
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        all_passed = False
    
    # Test 5: Full SwarmMamba
    print("\n[TEST 5] SwarmMamba end-to-end...")
    try:
        swarm = SwarmMamba(
            n_agents=4, d_model=32, n_layers=2, n_comm_rounds=2
        ).to(DEVICE)
        img = torch.randn(2, 3, 64, 64).to(DEVICE)
        logits, ind_logits, cons = swarm(img)
        assert logits.shape == (2, 2)
        assert len(ind_logits) == 4
        print(f"  ✓ SwarmMamba produces valid predictions")
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        all_passed = False
    
    # Test 6: Parameter count sanity
    print("\n[TEST 6] Parameter efficiency check...")
    try:
        swarm = SwarmMamba(n_agents=4, d_model=64, n_layers=2)
        single = SingleMamba(d_model=128, n_layers=4)
        
        swarm_params = sum(p.numel() for p in swarm.parameters())
        single_params = sum(p.numel() for p in single.parameters())
        
        ratio = swarm_params / single_params
        print(f"  Swarm params: {swarm_params:,}")
        print(f"  Single params: {single_params:,}")
        print(f"  Ratio: {ratio:.2f}x")
        
        # Swarm should be comparable (within 3x)
        assert ratio < 3.0, f"Swarm too large: {ratio:.2f}x baseline"
        print("  ✓ Parameter counts reasonable")
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        all_passed = False
    
    # Test 7: Gradient flow
    print("\n[TEST 7] Gradient flow through architecture...")
    try:
        swarm = SwarmMamba(n_agents=4, d_model=32).to(DEVICE)
        img = torch.randn(1, 3, 64, 64, requires_grad=True).to(DEVICE)
        logits, _, _ = swarm(img)
        loss = logits.sum()
        loss.backward()
        
        # Check gradients exist
        has_grad = any(p.grad is not None and p.grad.abs().sum() > 0 
                      for p in swarm.parameters())
        assert has_grad, "No gradients flowing"
        print("  ✓ Gradients flow correctly")
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        all_passed = False
    
    print("\n" + "=" * 70)
    if all_passed:
        print(" ALL TESTS PASSED ✓")
    else:
        print(" SOME TESTS FAILED ✗")
    print("=" * 70)
    
    return all_passed


# ============================================================================
# SECTION 10: MAIN EXPERIMENT
# ============================================================================

def main():
    """Main experimental pipeline for PoC validation."""
    
    print("\n" + "=" * 70)
    print(" SWARM-MAMBA: PROOF OF CONCEPT VALIDATION")
    print(" Asynchronous Parallel State Machines with Emergent Consensus")
    print("=" * 70)
    print(f"\n Device: {DEVICE}")
    print(f" Config: {CONFIG}")
    
    # Run test suite
    tests_passed = run_test_suite()
    if not tests_passed:
        print("\n❌ CRITICAL: Tests failed. Aborting experiment.")
        return "NO-GO", {}
    
    # Generate data
    print("\n" + "-" * 50)
    print(" DATA GENERATION")
    print("-" * 50)
    print(f" Generating synthetic medical imaging dataset...")
    
    X_train, y_train = generate_medical_imaging_dataset(CONFIG.n_train, CONFIG.img_size)
    X_test, y_test = generate_medical_imaging_dataset(CONFIG.n_test, CONFIG.img_size)
    
    print(f"  Training samples: {len(X_train)}")
    print(f"  Test samples: {len(X_test)}")
    print(f"  Image size: {CONFIG.img_size}x{CONFIG.img_size}x3")
    print(f"  Classes: {CONFIG.n_classes} (benign/malignant)")
    print(f"  Class balance: {(y_train == 0).sum().item()}/{(y_train == 1).sum().item()}")
    
    results = {}
    
    # ========================================
    # Experiment 1: Single Mamba Baseline
    # ========================================
    print("\n" + "-" * 50)
    print(" EXPERIMENT 1: Single Mamba (Baseline)")
    print("-" * 50)
    
    single_model = SingleMamba(d_model=128, n_layers=4)
    single_params = sum(p.numel() for p in single_model.parameters())
    print(f"  Parameters: {single_params:,}")
    
    print("  Training...")
    single_model = train_model(
        single_model, (X_train, y_train), CONFIG.epochs, is_swarm=False
    )
    
    print("  Evaluating...")
    single_results = evaluate_model(single_model, (X_test, y_test), is_swarm=False)
    results['single'] = single_results
    
    print(f"\n  Results:")
    print(f"    Accuracy: {single_results['accuracy']:.1%}")
    print(f"    Inference time: {single_results['inference_time_ms']:.2f} ms/sample")
    
    # ========================================
    # Experiment 2: Swarm-Mamba (k=4)
    # ========================================
    print("\n" + "-" * 50)
    print(" EXPERIMENT 2: Swarm-Mamba (k=4 agents)")
    print("-" * 50)
    
    swarm_model = SwarmMamba(
        n_agents=4, d_model=64, n_layers=2, n_comm_rounds=2
    )
    swarm_params = sum(p.numel() for p in swarm_model.parameters())
    print(f"  Parameters: {swarm_params:,}")
    print(f"  Parameter ratio: {swarm_params/single_params:.2f}x baseline")
    
    print("  Training...")
    swarm_model = train_model(
        swarm_model, (X_train, y_train), CONFIG.epochs, is_swarm=True
    )
    
    print("  Evaluating...")
    swarm_results = evaluate_model(swarm_model, (X_test, y_test), is_swarm=True)
    results['swarm_4'] = swarm_results
    
    print(f"\n  Results:")
    print(f"    Accuracy: {swarm_results['accuracy']:.1%}")
    print(f"    Inference time: {swarm_results['inference_time_ms']:.2f} ms/sample")
    print(f"    Consensus rate: {swarm_results['consensus_rate']:.1%}")
    
    # ========================================
    # Experiment 3: Agent Scaling
    # ========================================
    print("\n" + "-" * 50)
    print(" EXPERIMENT 3: Agent Scaling Analysis")
    print("-" * 50)
    
    for k in [2, 8]:
        print(f"\n  Testing k={k} agents...")
        model = SwarmMamba(
            n_agents=k, d_model=64, n_layers=2, n_comm_rounds=2
        )
        model = train_model(
            model, (X_train, y_train), CONFIG.epochs // 2, 
            is_swarm=True, verbose=False
        )
        res = evaluate_model(model, (X_test, y_test), is_swarm=True)
        results[f'swarm_{k}'] = res
        print(f"    k={k}: Acc={res['accuracy']:.1%}, "
              f"Time={res['inference_time_ms']:.2f}ms, "
              f"Consensus={res['consensus_rate']:.1%}")
    
    # ========================================
    # Decision Analysis
    # ========================================
    print("\n" + "=" * 70)
    print(" EVALUATION RESULTS & DECISION")
    print("=" * 70)
    
    # Calculate key metrics
    swarm_acc = results['swarm_4']['accuracy']
    single_acc = results['single']['accuracy']
    relative_accuracy = swarm_acc / single_acc if single_acc > 0 else 0
    
    swarm_time = results['swarm_4']['inference_time_ms']
    single_time = results['single']['inference_time_ms']
    speedup = single_time / swarm_time if swarm_time > 0 else 0
    
    consensus = results['swarm_4']['consensus_rate']
    
    # Decision criteria
    criteria = [
        (
            "Absolute Accuracy",
            swarm_acc >= CONFIG.accuracy_threshold,
            f"{swarm_acc:.1%}",
            f"≥{CONFIG.accuracy_threshold:.0%}"
        ),
        (
            "Relative Accuracy",
            relative_accuracy >= CONFIG.relative_accuracy_threshold,
            f"{relative_accuracy:.1%}",
            f"≥{CONFIG.relative_accuracy_threshold:.0%}"
        ),
        (
            "Inference Speed",
            speedup >= CONFIG.speedup_threshold,
            f"{speedup:.2f}x",
            f"≥{CONFIG.speedup_threshold:.1f}x"
        ),
        (
            "Agent Consensus",
            consensus >= CONFIG.consensus_threshold,
            f"{consensus:.1%}",
            f"≥{CONFIG.consensus_threshold:.0%}"
        ),
    ]
    
    # Print decision table
    print("\n┌" + "─" * 25 + "┬" + "─" * 12 + "┬" + "─" * 12 + "┬" + "─" * 10 + "┐")
    print(f"│{'Criterion':<25}│{'Achieved':^12}│{'Threshold':^12}│{'Status':^10}│")
    print("├" + "─" * 25 + "┼" + "─" * 12 + "┼" + "─" * 12 + "┼" + "─" * 10 + "┤")
    
    passes = 0
    for name, passed, achieved, threshold in criteria:
        status = "✓ PASS" if passed else "✗ FAIL"
        passes += int(passed)
        print(f"│{name:<25}│{achieved:^12}│{threshold:^12}│{status:^10}│")
    
    print("└" + "─" * 25 + "┴" + "─" * 12 + "┴" + "─" * 12 + "┴" + "─" * 10 + "┘")
    
    # Scaling analysis
    print("\n Agent Scaling Analysis:")
    scaling_results = [(k, results[f'swarm_{k}']['accuracy']) 
                       for k in [2, 4, 8] if f'swarm_{k}' in results]
    for k, acc in scaling_results:
        print(f"    k={k}: {acc:.1%}")
    
    # Check monotonicity
    accs = [r[1] for r in scaling_results]
    is_monotonic = all(accs[i] <= accs[i+1] for i in range(len(accs)-1))
    
    # ========================================
    # Final Verdict
    # ========================================
    print("\n" + "=" * 70)
    print(" STRATEGIC VERDICT")
    print("=" * 70)
    
    if passes >= 4:
        verdict = "GO"
        recommendation = "PROCEED TO FULL IMPLEMENTATION ON REAL DATA"
        icon = "🟢"
    elif passes >= 2:
        verdict = "IMPROVE"
        recommendation = "REFINE ARCHITECTURE - ADDRESS IDENTIFIED GAPS"
        icon = "🟡"
    else:
        verdict = "NO-GO"
        recommendation = "FUNDAMENTAL RETHINK REQUIRED"
        icon = "🔴"
    
    print(f"\n  {icon} FINAL VERDICT: {verdict}")
    print(f"  📊 Criteria Passed: {passes}/4")
    print(f"  📋 Recommendation: {recommendation}")
    
    # Key insights
    print("\n" + "-" * 50)
    print(" KEY INSIGHTS")
    print("-" * 50)
    
    insights = []
    
    if relative_accuracy >= 0.95:
        insights.append("✓ Distributed processing successfully maintains accuracy")
    else:
        insights.append(f"⚠ Accuracy gap ({(1-relative_accuracy)*100:.1f}%) - "
                       "consider deeper message passing")
    
    if consensus >= 0.7:
        insights.append("✓ High agent consensus indicates stable learned representations")
    else:
        insights.append("⚠ Low consensus - add stronger alignment regularization")
    
    if speedup < 1.0:
        insights.append("⚠ Communication overhead exceeds parallelization gain - "
                       "optimize message compression")
    
    if not is_monotonic:
        insights.append("⚠ Non-monotonic scaling suggests communication saturation")
    
    for insight in insights:
        print(f"  {insight}")
    
    # Technical recommendations
    print("\n" + "-" * 50)
    print(" TECHNICAL RECOMMENDATIONS FOR Q1")
    print("-" * 50)
    
    if verdict == "GO":
        recs = [
            "1. Implement true GPU parallelism via torch.multiprocessing",
            "2. Test on BreaKHis 400X with ResNet-pretrained patch embeddings",
            "3. Add curriculum learning: start with k=2, gradually increase",
            "4. Benchmark against MambaMIL and TransMIL baselines"
        ]
    elif verdict == "IMPROVE":
        recs = [
            "1. Reduce message passing overhead via sparse communication",
            "2. Implement asynchronous update (not all agents sync every round)",
            "3. Add contrastive loss between agent representations",
            "4. Experiment with hierarchical agent topology (tree vs. full)"
        ]
    else:
        recs = [
            "1. Revisit core hypothesis - is parallelism the right approach?",
            "2. Consider sequential multi-agent (not parallel)",
            "3. Try simpler consensus: just average logits"
        ]
    
    for rec in recs:
        print(f"  {rec}")
    
    print("\n" + "=" * 70)
    print(f" Q1 RESEARCH DECISION: {verdict}")
    print("=" * 70)
    
    return verdict, results


# ============================================================================
# EXECUTION
# ============================================================================

if __name__ == "__main__":
    verdict, results = main()


 SWARM-MAMBA: PROOF OF CONCEPT VALIDATION
 Asynchronous Parallel State Machines with Emergent Consensus

 Device: cuda
 Config: ExperimentConfig(seed=42, img_size=64, patch_size=8, n_classes=2, n_train=200, n_test=56, epochs=12, batch_size=32, lr=0.001, accuracy_threshold=0.65, relative_accuracy_threshold=0.9, speedup_threshold=1.0, consensus_threshold=0.6)

 RUNNING TEST SUITE

[TEST 1] SelectiveSSM forward pass...
  ✓ Output shape and values correct

[TEST 2] MicroMamba agent processing...
  ✓ Agent processes partition correctly

[TEST 3] Inter-agent message passing...
  ✓ Message passing modifies states correctly

[TEST 4] Emergent consensus...
  ✓ Consensus produces valid output (rate: 0.88)

[TEST 5] SwarmMamba end-to-end...
  ✓ SwarmMamba produces valid predictions

[TEST 6] Parameter efficiency check...
  Swarm params: 335,279
  Single params: 509,570
  Ratio: 0.66x
  ✓ Parameter counts reasonable

[TEST 7] Gradient flow through architecture...
  ✓ Gradients flow correctly

 AL

# experiments 

In [1]:
"""
═══════════════════════════════════════════════════════════════════════════════
SWARM-MAMBA: Q1 PUBLICATION CODE - PART 1 OF 2
═══════════════════════════════════════════════════════════════════════════════
Optimized for: Kaggle P100 GPU (16GB VRAM)
Contents:
  - T1: Comparative Benchmarking (Swarm-Mamba vs Vim vs ViT) - PARAMETER MATCHED
  - T2: Inter-Agent Consensus Analysis
  
Hardware Profile:
  - GPU: NVIDIA P100 (16GB)
  - Batch Size: 200 (optimized for memory)
  - Epochs: 5 (balanced speed/quality)
  
FAIR COMPARISON: All models ~520K parameters (within 1%)
═══════════════════════════════════════════════════════════════════════════════
"""

import os
import sys
import time
import gc
import warnings
import random
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from collections import defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score, 
    precision_score, recall_score, confusion_matrix,
    cohen_kappa_score
)
from scipy import stats

warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 1: CONFIGURATION (OPTIMIZED FOR P100 - ~520K PARAMS)
# ═══════════════════════════════════════════════════════════════════════════════

@dataclass
class Config:
    seeds: List[int] = field(default_factory=lambda: [42, 123, 2024])
    
    data_root: str = "/kaggle/input/breakhis-400x/BreaKHis 400X"
    img_size: int = 224
    n_classes: int = 2
    
    batch_size: int = 200         
    epochs: int = 5             
    lr: float = 3e-4
    weight_decay: float = 0.01
    
    # Swarm-Mamba configuration (target: ~524K params)
    d_model: int = 80
    n_layers: int = 2
    d_state: int = 16
    n_agents: int = 4
    n_comm_rounds: int = 2
    patch_size: int = 16
    
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    num_workers: int = 2
    pin_memory: bool = True
    
CONFIG = Config()

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

def get_memory_stats():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        max_allocated = torch.cuda.max_memory_allocated() / 1024**3
        return {'allocated': allocated, 'reserved': reserved, 'max': max_allocated}
    return {'allocated': 0, 'reserved': 0, 'max': 0}

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("═" * 70)
print("SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS - PART 1")
print("═" * 70)
print(f"Device: {CONFIG.device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"Seeds: {CONFIG.seeds}")
print(f"Epochs: {CONFIG.epochs}")
print(f"Batch Size: {CONFIG.batch_size}")
print(f"Image Size: {CONFIG.img_size}x{CONFIG.img_size}")
print("═" * 70)

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 2: DATASET
# ═══════════════════════════════════════════════════════════════════════════════

class BreaKHisDataset(Dataset):
    def __init__(self, root_dir: str, split: str = "train", transform=None):
        self.root_dir = Path(root_dir)
        self.split = split
        self.transform = transform
        self.samples = []
        self.labels = []
        self._load_data()
        
    def _load_data(self):
        split_dir = self.root_dir / self.split
        for class_idx, class_name in enumerate(["benign", "malignant"]):
            class_dir = split_dir / class_name
            if class_dir.exists():
                for img_path in sorted(class_dir.glob("*.png")):
                    self.samples.append(str(img_path))
                    self.labels.append(class_idx)
        self.labels = np.array(self.labels)
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img = Image.open(self.samples[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

def get_transforms(is_train: bool = True):
    if is_train:
        return transforms.Compose([
            transforms.Resize((CONFIG.img_size, CONFIG.img_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    return transforms.Compose([
        transforms.Resize((CONFIG.img_size, CONFIG.img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 3: SELECTIVE STATE SPACE MODEL (OPTIMIZED)
# ═══════════════════════════════════════════════════════════════════════════════

class SelectiveSSM(nn.Module):
    def __init__(self, d_model: int, d_state: int = 16, expand: int = 2):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_inner = d_model * expand
        self.dt_rank = max(d_model // 16, 1)
        
        self.in_proj = nn.Linear(d_model, self.d_inner * 2, bias=False)
        self.conv1d = nn.Conv1d(self.d_inner, self.d_inner, 4, padding=2, groups=self.d_inner)
        self.x_proj = nn.Linear(self.d_inner, self.dt_rank + d_state * 2, bias=False)
        self.dt_proj = nn.Linear(self.dt_rank, self.d_inner, bias=True)
        
        A = torch.arange(1, d_state + 1, dtype=torch.float32)
        self.A_log = nn.Parameter(torch.log(A))
        self.D = nn.Parameter(torch.ones(self.d_inner))
        
        self.out_proj = nn.Linear(self.d_inner, d_model, bias=False)
        self.norm = nn.LayerNorm(self.d_inner)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, L, _ = x.shape
        
        xz = self.in_proj(x)
        x_branch, z = xz.chunk(2, dim=-1)
        
        x_branch = x_branch.transpose(1, 2)
        x_branch = self.conv1d(x_branch)[:, :, :L]
        x_branch = x_branch.transpose(1, 2)
        x_branch = F.silu(x_branch)
        
        x_dbl = self.x_proj(x_branch)
        dt, B_param, C = torch.split(x_dbl, [self.dt_rank, self.d_state, self.d_state], dim=-1)
        dt = F.softplus(self.dt_proj(dt))
        
        A = -torch.exp(self.A_log)
        y = self._scan_optimized(x_branch, dt, A, B_param, C)
        
        y = self.norm(y)
        y = y * F.silu(z)
        return self.out_proj(y)
    
    def _scan_optimized(self, x, dt, A, B, C):
        B_size, L, D = x.shape
        
        chunk_size = 32
        outputs = []
        h = torch.zeros(B_size, D, self.d_state, device=x.device, dtype=x.dtype)
        
        for start in range(0, L, chunk_size):
            end = min(start + chunk_size, L)
            chunk_out = []
            
            for t in range(start, end):
                dt_t = dt[:, t, :].unsqueeze(-1)
                dA = torch.exp(A.view(1, 1, -1) * dt_t)
                dB = dt_t * B[:, t, :].unsqueeze(1)
                h = dA * h + dB * x[:, t, :].unsqueeze(-1)
                y_t = (h * C[:, t, :].unsqueeze(1)).sum(-1) + self.D * x[:, t, :]
                chunk_out.append(y_t)
            
            outputs.extend(chunk_out)
        
        return torch.stack(outputs, dim=1)

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 4: MICRO-MAMBA AGENT
# ═══════════════════════════════════════════════════════════════════════════════

class MicroMamba(nn.Module):
    def __init__(self, d_model: int, n_layers: int = 2, d_state: int = 16):
        super().__init__()
        self.d_model = d_model
        self.blocks = nn.ModuleList([
            nn.ModuleDict({
                'norm': nn.LayerNorm(d_model),
                'ssm': SelectiveSSM(d_model, d_state),
                'drop': nn.Dropout(0.1)
            }) for _ in range(n_layers)
        ])
        self.agent_embed = nn.Parameter(torch.randn(1, 1, d_model) * 0.02)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B = x.shape[0]
        x = torch.cat([self.agent_embed.expand(B, -1, -1), x], dim=1)
        
        for block in self.blocks:
            res = x
            x = block['norm'](x)
            x = block['ssm'](x)
            x = block['drop'](x)
            x = res + x
            
        return x[:, 1:, :]

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 5: MESSAGE PASSING & CONSENSUS
# ═══════════════════════════════════════════════════════════════════════════════

class MessagePassing(nn.Module):
    def __init__(self, d_model: int, n_agents: int, n_heads: int = 4):
        super().__init__()
        self.n_agents = n_agents
        self.d_model = d_model
        self.d_compress = d_model // 2
        
        self.compress = nn.Sequential(
            nn.Linear(d_model, self.d_compress),
            nn.GELU(),
            nn.LayerNorm(self.d_compress)
        )
        self.decompress = nn.Linear(self.d_compress, d_model)
        
        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=0.1, batch_first=True)
        self.alpha = nn.Parameter(torch.tensor(0.3))
        
    def forward(self, states: List[torch.Tensor]) -> List[torch.Tensor]:
        B = states[0].shape[0]
        
        summaries = torch.stack([self.compress(s.mean(dim=1)) for s in states], dim=1)
        decompressed = self.decompress(summaries)
        
        attn_out, _ = self.attn(decompressed, decompressed, decompressed)
        
        alpha = torch.sigmoid(self.alpha)
        updated = []
        for i, s in enumerate(states):
            msg = attn_out[:, i:i+1, :].expand(-1, s.shape[1], -1)
            updated.append(s + alpha * msg)
            
        return updated

class Consensus(nn.Module):
    def __init__(self, d_model: int, n_agents: int, n_classes: int):
        super().__init__()
        self.n_agents = n_agents
        self.importance = nn.Parameter(torch.ones(n_agents) / n_agents)
        
        self.agent_heads = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm(d_model),
                nn.Linear(d_model, d_model // 2),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(d_model // 2, n_classes)
            ) for _ in range(n_agents)
        ])
        
        self.consensus_head = nn.Sequential(
            nn.LayerNorm(d_model * n_agents),
            nn.Linear(d_model * n_agents, d_model),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(d_model, n_classes)
        )
        
    def forward(self, states: List[torch.Tensor]) -> Tuple[torch.Tensor, List[torch.Tensor], Dict]:
        pooled = [s.mean(dim=1) for s in states]
        individual = [head(p) for head, p in zip(self.agent_heads, pooled)]
        
        concat = torch.cat(pooled, dim=-1)
        consensus_logits = self.consensus_head(concat)
        
        with torch.no_grad():
            preds = torch.stack([torch.argmax(l, dim=-1) for l in individual], dim=1)
            mode_pred = torch.mode(preds, dim=1).values
            agreement = (preds == mode_pred.unsqueeze(1)).float().mean()
            
            kappas = []
            for i in range(len(individual)):
                for j in range(i+1, len(individual)):
                    p_i = torch.argmax(individual[i], dim=-1).cpu().numpy()
                    p_j = torch.argmax(individual[j], dim=-1).cpu().numpy()
                    if len(np.unique(p_i)) > 1 and len(np.unique(p_j)) > 1:
                        try:
                            kappas.append(cohen_kappa_score(p_i, p_j))
                        except:
                            pass
            
        metrics = {
            'consensus_rate': agreement.item(),
            'mean_kappa': np.mean(kappas) if kappas else 1.0
        }
        
        return consensus_logits, individual, metrics

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 6: SWARM-MAMBA ARCHITECTURE
# ═══════════════════════════════════════════════════════════════════════════════

class SwarmMamba(nn.Module):
    def __init__(
        self,
        img_size: int = 224,
        patch_size: int = 16,
        in_chans: int = 3,
        n_agents: int = 4,
        d_model: int = 80,
        n_layers: int = 2,
        n_classes: int = 2,
        n_comm_rounds: int = 2,
        d_state: int = 16,
        use_communication: bool = True
    ):
        super().__init__()
        self.n_agents = n_agents
        self.n_comm_rounds = n_comm_rounds
        self.use_communication = use_communication
        
        self.n_patches = (img_size // patch_size) ** 2
        
        self.patch_embed = nn.Conv2d(in_chans, d_model, patch_size, patch_size)
        self.pos_embed = nn.Parameter(torch.randn(1, self.n_patches, d_model) * 0.02)
        
        self.agents = nn.ModuleList([
            MicroMamba(d_model, n_layers, d_state) for _ in range(n_agents)
        ])
        
        self.message_passing = MessagePassing(d_model, n_agents)
        self.consensus = Consensus(d_model, n_agents, n_classes)
        
    def partition(self, x: torch.Tensor) -> List[torch.Tensor]:
        B, N, D = x.shape
        per_agent = N // self.n_agents
        return [x[:, i*per_agent:(i+1)*per_agent if i < self.n_agents-1 else N, :].clone() 
                for i in range(self.n_agents)]
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, List[torch.Tensor], Dict]:
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        x = x + self.pos_embed
        
        partitions = self.partition(x)
        states = [agent(part) for agent, part in zip(self.agents, partitions)]
        
        for _ in range(self.n_comm_rounds):
            if self.use_communication:
                states = self.message_passing(states)
            states = [agent(s) for agent, s in zip(self.agents, states)]
        
        return self.consensus(states)

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 7: BASELINE MODELS (PARAMETER-MATCHED ~520K)
# ═══════════════════════════════════════════════════════════════════════════════

class VisionMamba(nn.Module):
    """
    Vision Mamba baseline - PARAMETER MATCHED to Swarm-Mamba (~522K)
    Config: d_model=82, n_layers=9
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, d_model=82, 
                 n_layers=9, n_classes=2, d_state=16):
        super().__init__()
        self.n_patches = (img_size // patch_size) ** 2
        
        self.patch_embed = nn.Conv2d(in_chans, d_model, patch_size, patch_size)
        self.pos_embed = nn.Parameter(torch.randn(1, self.n_patches, d_model) * 0.02)
        
        self.blocks = nn.ModuleList([
            nn.ModuleDict({
                'norm': nn.LayerNorm(d_model),
                'ssm': SelectiveSSM(d_model, d_state)
            }) for _ in range(n_layers)
        ])
        
        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(d_model // 2, n_classes)
        )
        
    def forward(self, x):
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        x = x + self.pos_embed
        
        for block in self.blocks:
            x = x + block['ssm'](block['norm'](x))
        
        return self.head(x.mean(dim=1))

class SimpleViT(nn.Module):
    """
    Vision Transformer baseline - PARAMETER MATCHED to Swarm-Mamba (~521K)
    Config: d_model=102, n_layers=5, mlp_ratio=2, n_heads=6
    Note: 102 is divisible by 6 (102/6=17)
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, d_model=102,
                 n_layers=5, n_heads=6, n_classes=2, mlp_ratio=2):
        super().__init__()
        self.n_patches = (img_size // patch_size) ** 2
        
        self.patch_embed = nn.Conv2d(in_chans, d_model, patch_size, patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model) * 0.02)
        self.pos_embed = nn.Parameter(torch.randn(1, self.n_patches + 1, d_model) * 0.02)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=n_heads, 
            dim_feedforward=d_model * mlp_ratio,
            dropout=0.1, 
            activation='gelu', 
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, n_layers)
        
        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, n_classes)
        )
        
    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        cls = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls, x], dim=1)
        x = x + self.pos_embed
        x = self.encoder(x)
        return self.head(x[:, 0])

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 8: PARAMETER VERIFICATION
# ═══════════════════════════════════════════════════════════════════════════════

def verify_parameter_matching():
    """Verify all models have approximately equal parameters (~520K, within 1%)"""
    print("\n" + "═" * 70)
    print("PARAMETER VERIFICATION FOR FAIR COMPARISON")
    print("Target: ~520K parameters per model (within 1%)")
    print("═" * 70)
    
    models = {
        'Swarm-Mamba': SwarmMamba(
            img_size=CONFIG.img_size,
            patch_size=CONFIG.patch_size,
            n_agents=CONFIG.n_agents,
            d_model=CONFIG.d_model,       # 80
            n_layers=CONFIG.n_layers,      # 2
            n_classes=CONFIG.n_classes,
            n_comm_rounds=CONFIG.n_comm_rounds,
            d_state=CONFIG.d_state
        ),
        'Vision-Mamba': VisionMamba(
            img_size=CONFIG.img_size,
            patch_size=CONFIG.patch_size,
            d_model=82,       # Tuned for ~522K
            n_layers=9,       # Tuned for ~522K
            n_classes=CONFIG.n_classes,
            d_state=CONFIG.d_state
        ),
        'ViT': SimpleViT(
            img_size=CONFIG.img_size,
            patch_size=CONFIG.patch_size,
            d_model=102,      # Tuned for ~521K
            n_layers=5,       # Tuned for ~521K
            n_heads=6,        # 102 / 6 = 17 (must divide evenly)
            n_classes=CONFIG.n_classes,
            mlp_ratio=2
        )
    }
    
    param_counts = {}
    print(f"\n{'Model':<20} {'Parameters':>15} {'Relative':>12}")
    print("─" * 50)
    
    for name, model in models.items():
        params = count_parameters(model)
        param_counts[name] = params
    
    base_params = param_counts['Swarm-Mamba']
    
    for name, params in param_counts.items():
        relative = params / base_params
        print(f"{name:<20} {params:>15,} {relative:>11.3f}x")
    
    min_params = min(param_counts.values())
    max_params = max(param_counts.values())
    ratio = max_params / min_params
    
    print("─" * 50)
    print(f"Parameter Ratio (max/min): {ratio:.4f}x")
    print(f"Target: < 1.05x for fair comparison")
    
    if ratio <= 1.01:
        print("✓ EXCELLENT: All models within 1% parameter range")
    elif ratio <= 1.05:
        print("✓ FAIR COMPARISON: All models within 5% parameter range")
    elif ratio <= 1.10:
        print("⚠ ACCEPTABLE: All models within 10% parameter range")
    else:
        print("✗ WARNING: Parameter difference exceeds 10%")
    
    return param_counts

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 9: TRAINING & EVALUATION WITH TIMING
# ═══════════════════════════════════════════════════════════════════════════════

class Timer:
    def __init__(self):
        self.times = {}
        
    def start(self, name):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        self.times[name] = {'start': time.perf_counter()}
        
    def stop(self, name):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        self.times[name]['end'] = time.perf_counter()
        self.times[name]['duration'] = self.times[name]['end'] - self.times[name]['start']
        return self.times[name]['duration']
    
    def get(self, name):
        return self.times.get(name, {}).get('duration', 0)

def train_epoch(model, loader, optimizer, criterion, device, is_swarm=False):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        if is_swarm:
            logits, individual, _ = model(batch_x)
            loss = criterion(logits, batch_y)
            for ind in individual:
                loss += 0.1 * criterion(ind, batch_y)
        else:
            logits = model(batch_x)
            loss = criterion(logits, batch_y)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item() * batch_x.size(0)
        preds = torch.argmax(logits, dim=-1)
        correct += (preds == batch_y).sum().item()
        total += batch_y.size(0)
    
    return total_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader, device, is_swarm=False):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    consensus_data = {'rates': [], 'kappas': []}
    
    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
        
        if is_swarm:
            logits, individual, metrics = model(batch_x)
            consensus_data['rates'].append(metrics['consensus_rate'])
            consensus_data['kappas'].append(metrics['mean_kappa'])
        else:
            logits = model(batch_x)
        
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(logits, dim=-1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    results = {
        'accuracy': accuracy_score(all_labels, all_preds),
        'auc': roc_auc_score(all_labels, all_probs) if len(np.unique(all_labels)) > 1 else 0.5,
        'f1': f1_score(all_labels, all_preds, average='weighted'),
        'precision': precision_score(all_labels, all_preds, average='weighted', zero_division=0),
        'recall': recall_score(all_labels, all_preds, average='weighted', zero_division=0),
        'sensitivity': recall_score(all_labels, all_preds, pos_label=1, zero_division=0),
        'specificity': recall_score(all_labels, all_preds, pos_label=0, zero_division=0)
    }
    
    if consensus_data['rates']:
        results['consensus_rate'] = np.mean(consensus_data['rates'])
        results['mean_kappa'] = np.mean(consensus_data['kappas'])
    
    return results

@torch.no_grad()
def measure_inference_time(model, loader, device, is_swarm=False, n_warmup=5, n_runs=50):
    model.eval()
    
    batch_x, _ = next(iter(loader))
    batch_x = batch_x.to(device)
    
    for _ in range(n_warmup):
        if is_swarm:
            _ = model(batch_x)
        else:
            _ = model(batch_x)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        if is_swarm:
            _ = model(batch_x)
        else:
            _ = model(batch_x)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        times.append(time.perf_counter() - start)
    
    return {
        'mean_ms': np.mean(times) * 1000,
        'std_ms': np.std(times) * 1000,
        'per_sample_ms': np.mean(times) * 1000 / batch_x.size(0),
        'throughput': batch_x.size(0) / np.mean(times)
    }

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 10: EXPERIMENT T1 - COMPARATIVE BENCHMARKING
# ═══════════════════════════════════════════════════════════════════════════════

def run_T1_benchmarking():
    print("\n" + "═" * 70)
    print("EXPERIMENT T1: ACCURACY VS COMPLEXITY BENCHMARKING")
    print("(PARAMETER-MATCHED COMPARISON ~520K params each)")
    print("═" * 70)
    
    timer = Timer()
    timer.start('T1_total')
    
    train_ds = BreaKHisDataset(CONFIG.data_root, "train", get_transforms(True))
    test_ds = BreaKHisDataset(CONFIG.data_root, "test", get_transforms(False))
    
    print(f"\nDataset Statistics:")
    print(f"  Training samples: {len(train_ds)}")
    print(f"  Test samples: {len(test_ds)}")
    print(f"  Train class distribution: Benign={np.sum(train_ds.labels==0)}, Malignant={np.sum(train_ds.labels==1)}")
    print(f"  Test class distribution: Benign={np.sum(test_ds.labels==0)}, Malignant={np.sum(test_ds.labels==1)}")
    
    # Parameter-matched model configurations (~520K each)
    models_config = {
        'Swarm-Mamba': lambda: SwarmMamba(
            img_size=CONFIG.img_size,
            patch_size=CONFIG.patch_size,
            n_agents=CONFIG.n_agents,
            d_model=CONFIG.d_model,        # 80
            n_layers=CONFIG.n_layers,       # 2
            n_classes=CONFIG.n_classes,
            n_comm_rounds=CONFIG.n_comm_rounds,
            d_state=CONFIG.d_state
        ),
        'Vision-Mamba': lambda: VisionMamba(
            img_size=CONFIG.img_size,
            patch_size=CONFIG.patch_size,
            d_model=82,        # Tuned for ~522K
            n_layers=9,        # Tuned for ~522K
            n_classes=CONFIG.n_classes,
            d_state=CONFIG.d_state
        ),
        'ViT': lambda: SimpleViT(
            img_size=CONFIG.img_size,
            patch_size=CONFIG.patch_size,
            d_model=102,       # Tuned for ~521K
            n_layers=5,        # Tuned for ~521K
            n_heads=6,         # 102/6=17
            n_classes=CONFIG.n_classes,
            mlp_ratio=2
        )
    }
    
    all_results = {name: {
        'auc': [], 'acc': [], 'f1': [], 'precision': [], 'recall': [],
        'sensitivity': [], 'specificity': [],
        'params': 0, 'train_time': [], 'inference_time': []
    } for name in models_config}
    
    # Verify and display parameter counts
    print("\n" + "─" * 50)
    print("MODEL PARAMETER COUNTS (MATCHED ~520K)")
    print("─" * 50)
    for name, model_fn in models_config.items():
        model = model_fn()
        params = count_parameters(model)
        all_results[name]['params'] = params
        print(f"  {name:<15}: {params:>10,} parameters")
        del model
    
    params_list = [all_results[name]['params'] for name in models_config]
    ratio = max(params_list) / min(params_list)
    print("─" * 50)
    print(f"  Max/Min Ratio: {ratio:.4f}x {'✓' if ratio <= 1.05 else '⚠'}")
    print("─" * 50)
    
    for seed_idx, seed in enumerate(CONFIG.seeds):
        print(f"\n{'─' * 70}")
        print(f"SEED {seed_idx+1}/{len(CONFIG.seeds)}: {seed}")
        print('─' * 70)
        set_seed(seed)
        
        train_loader = DataLoader(
            train_ds, batch_size=CONFIG.batch_size, shuffle=True,
            num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory
        )
        test_loader = DataLoader(
            test_ds, batch_size=CONFIG.batch_size, shuffle=False,
            num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory
        )
        
        for model_name, model_fn in models_config.items():
            print(f"\n  Model: {model_name}")
            print(f"  {'-' * 40}")
            
            clear_memory()
            
            model = model_fn().to(CONFIG.device)
            params = count_parameters(model)
            
            print(f"  Parameters: {params:,}")
            
            is_swarm = 'Swarm' in model_name
            optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG.lr, weight_decay=CONFIG.weight_decay)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CONFIG.epochs)
            criterion = nn.CrossEntropyLoss()
            
            timer.start(f'{model_name}_train')
            
            for epoch in range(CONFIG.epochs):
                epoch_start = time.perf_counter()
                train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, CONFIG.device, is_swarm)
                scheduler.step()
                epoch_time = time.perf_counter() - epoch_start
                
                print(f"  Epoch {epoch+1:2d}/{CONFIG.epochs} | Loss: {train_loss:.4f} | "
                      f"Acc: {train_acc:.4f} | Time: {epoch_time:.1f}s")
            
            train_time = timer.stop(f'{model_name}_train')
            all_results[model_name]['train_time'].append(train_time)
            
            results = evaluate(model, test_loader, CONFIG.device, is_swarm)
            
            inf_time = measure_inference_time(model, test_loader, CONFIG.device, is_swarm)
            all_results[model_name]['inference_time'].append(inf_time['per_sample_ms'])
            
            for key in ['auc', 'acc', 'f1', 'precision', 'recall', 'sensitivity', 'specificity']:
                if key == 'acc':
                    all_results[model_name][key].append(results['accuracy'])
                else:
                    all_results[model_name][key].append(results.get(key, 0))
            
            mem = get_memory_stats()
            print(f"\n  Test Results:")
            print(f"    AUC: {results['auc']:.4f}")
            print(f"    Accuracy: {results['accuracy']:.4f}")
            print(f"    F1: {results['f1']:.4f}")
            print(f"    Sensitivity: {results['sensitivity']:.4f}")
            print(f"    Specificity: {results['specificity']:.4f}")
            print(f"    Train Time: {train_time:.1f}s")
            print(f"    Inference: {inf_time['per_sample_ms']:.2f} ms/sample")
            print(f"    Throughput: {inf_time['throughput']:.1f} samples/s")
            print(f"    GPU Memory: {mem['max']:.2f} GB")
            
            del model
            clear_memory()
    
    t1_time = timer.stop('T1_total')
    
    print("\n" + "═" * 70)
    print("T1 RESULTS SUMMARY (PARAMETER-MATCHED COMPARISON ~520K)")
    print("═" * 70)
    
    print(f"\n{'Model':<15} {'Params':>10} {'AUC':>15} {'Accuracy':>15} {'F1':>15}")
    print("─" * 70)
    
    for name, data in all_results.items():
        auc_mean, auc_std = np.mean(data['auc']), np.std(data['auc'])
        acc_mean, acc_std = np.mean(data['acc']), np.std(data['acc'])
        f1_mean, f1_std = np.mean(data['f1']), np.std(data['f1'])
        
        print(f"{name:<15} {data['params']:>10,} "
              f"{auc_mean:.4f}±{auc_std:.4f} "
              f"{acc_mean:.4f}±{acc_std:.4f} "
              f"{f1_mean:.4f}±{f1_std:.4f}")
    
    print("\n" + "─" * 70)
    print("TIMING COMPARISON")
    print("─" * 70)
    print(f"\n{'Model':<15} {'Train Time (s)':>18} {'Inference (ms/sample)':>25}")
    print("─" * 60)
    
    for name, data in all_results.items():
        train_mean = np.mean(data['train_time'])
        train_std = np.std(data['train_time'])
        inf_mean = np.mean(data['inference_time'])
        inf_std = np.std(data['inference_time'])
        print(f"{name:<15} {train_mean:>8.1f}±{train_std:>5.1f}      {inf_mean:>10.2f}±{inf_std:>5.2f}")
    
    print("\n" + "─" * 70)
    print("STATISTICAL SIGNIFICANCE (Paired t-test: Swarm-Mamba vs Others)")
    print("─" * 70)
    
    swarm_auc = all_results['Swarm-Mamba']['auc']
    for name in ['Vision-Mamba', 'ViT']:
        other_auc = all_results[name]['auc']
        if len(swarm_auc) >= 3 and len(other_auc) >= 3:
            t_stat, p_val = stats.ttest_rel(swarm_auc, other_auc)
            diff = np.mean(swarm_auc) - np.mean(other_auc)
            effect = diff / np.std(other_auc) if np.std(other_auc) > 0 else 0
            sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
            print(f"  vs {name:<15}: Δ={diff:+.4f}, t={t_stat:>7.3f}, p={p_val:.4f} {sig}, Cohen's d={effect:.3f}")
    
    print(f"\nT1 Total Execution Time: {t1_time/60:.2f} minutes")
    
    return all_results

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 11: EXPERIMENT T2 - CONSENSUS ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════════

def run_T2_consensus_analysis():
    print("\n" + "═" * 70)
    print("EXPERIMENT T2: INTER-AGENT CONSENSUS ANALYSIS")
    print("═" * 70)
    
    timer = Timer()
    timer.start('T2_total')
    
    consensus_results = {
        'overall_rate': [],
        'overall_kappa': [],
        'correct_consensus': [],
        'incorrect_consensus': [],
        'agent_accuracies': []
    }
    
    for seed_idx, seed in enumerate(CONFIG.seeds):
        print(f"\n{'─' * 50}")
        print(f"SEED {seed_idx+1}/{len(CONFIG.seeds)}: {seed}")
        print('─' * 50)
        set_seed(seed)
        clear_memory()
        
        train_ds = BreaKHisDataset(CONFIG.data_root, "train", get_transforms(True))
        test_ds = BreaKHisDataset(CONFIG.data_root, "test", get_transforms(False))
        
        train_loader = DataLoader(train_ds, batch_size=CONFIG.batch_size, shuffle=True,
                                 num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory)
        test_loader = DataLoader(test_ds, batch_size=CONFIG.batch_size, shuffle=False,
                                num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory)
        
        model = SwarmMamba(
            img_size=CONFIG.img_size,
            patch_size=CONFIG.patch_size,
            n_agents=CONFIG.n_agents,
            d_model=CONFIG.d_model,
            n_layers=CONFIG.n_layers,
            n_classes=CONFIG.n_classes,
            n_comm_rounds=CONFIG.n_comm_rounds
        ).to(CONFIG.device)
        
        optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG.lr, weight_decay=CONFIG.weight_decay)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CONFIG.epochs)
        criterion = nn.CrossEntropyLoss()
        
        print("  Training Swarm-Mamba...")
        for epoch in range(CONFIG.epochs):
            train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, CONFIG.device, is_swarm=True)
            scheduler.step()
            print(f"    Epoch {epoch+1:2d}/{CONFIG.epochs} | Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
        
        print("  Analyzing consensus patterns...")
        model.eval()
        
        batch_rates = []
        batch_kappas = []
        correct_rates = []
        incorrect_rates = []
        agent_preds_all = [[] for _ in range(CONFIG.n_agents)]
        all_labels = []
        
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x, batch_y = batch_x.to(CONFIG.device), batch_y.to(CONFIG.device)
                logits, individual, metrics = model(batch_x)
                
                batch_rates.append(metrics['consensus_rate'])
                batch_kappas.append(metrics['mean_kappa'])
                
                preds = torch.argmax(logits, dim=-1)
                correct_mask = (preds == batch_y)
                
                ind_preds = torch.stack([torch.argmax(l, dim=-1) for l in individual], dim=1)
                
                for i in range(len(batch_y)):
                    sample_preds = ind_preds[i]
                    mode_pred = torch.mode(sample_preds).values
                    sample_consensus = (sample_preds == mode_pred).float().mean().item()
                    
                    if correct_mask[i]:
                        correct_rates.append(sample_consensus)
                    else:
                        incorrect_rates.append(sample_consensus)
                
                for a in range(CONFIG.n_agents):
                    agent_preds_all[a].extend(torch.argmax(individual[a], dim=-1).cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
        
        agent_accs = []
        all_labels = np.array(all_labels)
        for a in range(CONFIG.n_agents):
            agent_preds = np.array(agent_preds_all[a])
            agent_acc = accuracy_score(all_labels, agent_preds)
            agent_accs.append(agent_acc)
        
        consensus_results['overall_rate'].append(np.mean(batch_rates))
        consensus_results['overall_kappa'].append(np.mean(batch_kappas))
        consensus_results['correct_consensus'].append(np.mean(correct_rates) if correct_rates else 0)
        consensus_results['incorrect_consensus'].append(np.mean(incorrect_rates) if incorrect_rates else 0)
        consensus_results['agent_accuracies'].append(agent_accs)
        
        print(f"\n  Seed {seed} Results:")
        print(f"    Overall Consensus Rate: {np.mean(batch_rates):.4f}")
        print(f"    Mean Inter-Agent Kappa: {np.mean(batch_kappas):.4f}")
        print(f"    Consensus on Correct: {np.mean(correct_rates):.4f}" if correct_rates else "    Consensus on Correct: N/A")
        print(f"    Consensus on Incorrect: {np.mean(incorrect_rates):.4f}" if incorrect_rates else "    Consensus on Incorrect: N/A")
        print(f"    Agent Accuracies: {[f'{a:.4f}' for a in agent_accs]}")
        
        del model
        clear_memory()
    
    t2_time = timer.stop('T2_total')
    
    print("\n" + "═" * 70)
    print("T2 RESULTS SUMMARY")
    print("═" * 70)
    
    print(f"\nOverall Consensus Rate: {np.mean(consensus_results['overall_rate']):.4f} ± "
          f"{np.std(consensus_results['overall_rate']):.4f}")
    print(f"Mean Inter-Agent Kappa: {np.mean(consensus_results['overall_kappa']):.4f} ± "
          f"{np.std(consensus_results['overall_kappa']):.4f}")
    
    print(f"\nConsensus on CORRECT predictions: {np.mean(consensus_results['correct_consensus']):.4f} ± "
          f"{np.std(consensus_results['correct_consensus']):.4f}")
    print(f"Consensus on INCORRECT predictions: {np.mean(consensus_results['incorrect_consensus']):.4f} ± "
          f"{np.std(consensus_results['incorrect_consensus']):.4f}")
    
    threshold_met = np.mean(consensus_results['correct_consensus']) > 0.85
    print(f"\nT2 Success Criterion (Consensus > 85% on correct): {'PASSED ✓' if threshold_met else 'NOT MET ✗'}")
    
    if len(consensus_results['correct_consensus']) >= 2 and len(consensus_results['incorrect_consensus']) >= 2:
        t_stat, p_val = stats.ttest_ind(
            consensus_results['correct_consensus'],
            consensus_results['incorrect_consensus']
        )
        print(f"\nConsensus Difference Test:")
        print(f"  t-statistic: {t_stat:.4f}")
        print(f"  p-value: {p_val:.4f}")
        sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
        print(f"  Significance: {sig if sig else 'Not significant'}")
    
    print("\n" + "─" * 50)
    print("AGENT SPECIALIZATION ANALYSIS")
    print("─" * 50)
    
    mean_agent_accs = np.mean(consensus_results['agent_accuracies'], axis=0)
    std_agent_accs = np.std(consensus_results['agent_accuracies'], axis=0)
    
    for a in range(CONFIG.n_agents):
        print(f"  Agent {a+1} Accuracy: {mean_agent_accs[a]:.4f} ± {std_agent_accs[a]:.4f}")
    
    diversity = np.std(mean_agent_accs)
    print(f"\n  Agent Diversity (σ of accuracies): {diversity:.4f}")
    print(f"  Interpretation: {'Low diversity - agents similar' if diversity < 0.02 else 'Moderate diversity' if diversity < 0.05 else 'High diversity - agents specialized'}")
    
    print(f"\nT2 Total Execution Time: {t2_time/60:.2f} minutes")
    
    return consensus_results

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 12: MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    print("\n" + "█" * 70)
    print("█" + " " * 68 + "█")
    print("█" + "   SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS".center(68) + "█")
    print("█" + "   PART 1: Benchmarking & Consensus Analysis".center(68) + "█")
    print("█" + "   (PARAMETER-MATCHED FAIR COMPARISON ~520K)".center(68) + "█")
    print("█" + " " * 68 + "█")
    print("█" * 70)
    
    total_timer = Timer()
    total_timer.start('total')
    
    print("\n" + "─" * 70)
    print("SYSTEM INFORMATION")
    print("─" * 70)
    print(f"PyTorch Version: {torch.__version__}")
    print(f"CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    data_path = Path(CONFIG.data_root)
    if not data_path.exists():
        print(f"\nERROR: Data not found at {CONFIG.data_root}")
        print("Please ensure BreaKHis 400X dataset is available.")
        sys.exit(1)
    
    # Verify parameter matching before experiments
    param_counts = verify_parameter_matching()
    
    print("\n" + "█" * 70)
    t1_results = run_T1_benchmarking()
    
    print("\n" + "█" * 70)
    t2_results = run_T2_consensus_analysis()
    
    total_time = total_timer.stop('total')
    final_mem = get_memory_stats()
    
    print("\n" + "═" * 70)
    print("PART 1 EXECUTION COMPLETE")
    print("═" * 70)
    print(f"\nTotal Execution Time: {total_time/60:.2f} minutes ({total_time:.1f} seconds)")
    print(f"Peak GPU Memory Used: {final_mem['max']:.2f} GB")
    
    print("\n" + "─" * 70)
    print("EXPERIMENTS COMPLETED IN PART 1:")
    print("─" * 70)
    print("  ✓ Parameter Verification (all models ~520K, within 1%)")
    print("  ✓ T1: Comparative Benchmarking (3 models × 3 seeds)")
    print("  ✓ T2: Inter-Agent Consensus Analysis (3 seeds)")
    
    print("\n" + "─" * 70)
    print("PROCEED TO PART 2 FOR:")
    print("─" * 70)
    print("  → T3: Noise Robustness Testing")
    print("  → T4: Communication Ablation Study")
    print("  → T5: Inference Latency Benchmarking")
    print("  → Final Statistical Summary")
    print("  → Publication-Ready Tables")
    print("═" * 70)

══════════════════════════════════════════════════════════════════════
SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS - PART 1
══════════════════════════════════════════════════════════════════════
Device: cuda
GPU: Tesla P100-PCIE-16GB
GPU Memory: 15.9 GB
Seeds: [42, 123, 2024]
Epochs: 5
Batch Size: 200
Image Size: 224x224
══════════════════════════════════════════════════════════════════════

██████████████████████████████████████████████████████████████████████
█                                                                    █
█                SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS             █
█               PART 1: Benchmarking & Consensus Analysis            █
█               (PARAMETER-MATCHED FAIR COMPARISON ~520K)            █
█                                                                    █
██████████████████████████████████████████████████████████████████████

──────────────────────────────────────────────────────────────────────
SYSTEM INFORMATION
─────────────────────

In [2]:
"""
═══════════════════════════════════════════════════════════════════════════════
SWARM-MAMBA: Q1 PUBLICATION CODE - PART 2 OF 2
═══════════════════════════════════════════════════════════════════════════════
Optimized for: Kaggle P100 GPU (16GB VRAM)
Contents:
  - T3: Noise Robustness Testing
  - T4: Communication Ablation Study
  - T5: Inference Latency Benchmarking
  - Final Statistical Summary
  - Publication-Ready Tables
  
Hardware Profile:
  - GPU: NVIDIA P100 (16GB)
  - Batch Size: 200 (optimized for memory)
  - Epochs: 5 (balanced speed/quality)
═══════════════════════════════════════════════════════════════════════════════
"""

import os
import sys
import time
import gc
import warnings
import random
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from collections import defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score, 
    precision_score, recall_score, confusion_matrix,
    cohen_kappa_score
)
from scipy import stats

warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════════════
# IMPORT CONFIGURATION AND MODELS FROM PART 1
# ═══════════════════════════════════════════════════════════════════════════════

@dataclass
class Config:
    seeds: List[int] = field(default_factory=lambda: [42, 123, 2024])
    data_root: str = "/kaggle/input/breakhis-400x/BreaKHis 400X"
    img_size: int = 224
    n_classes: int = 2
    batch_size: int = 200
    epochs: int = 5
    lr: float = 3e-4
    weight_decay: float = 0.01
    d_model: int = 80
    n_layers: int = 2
    d_state: int = 16
    n_agents: int = 4
    n_comm_rounds: int = 2
    patch_size: int = 16
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    num_workers: int = 2
    pin_memory: bool = True

CONFIG = Config()

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

def get_memory_stats():
    if torch.cuda.is_available():
        return {
            'allocated': torch.cuda.memory_allocated() / 1024**3,
            'reserved': torch.cuda.memory_reserved() / 1024**3,
            'max': torch.cuda.max_memory_allocated() / 1024**3
        }
    return {'allocated': 0, 'reserved': 0, 'max': 0}

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("═" * 70)
print("SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS - PART 2")
print("═" * 70)
print(f"Device: {CONFIG.device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("═" * 70)

# ═══════════════════════════════════════════════════════════════════════════════
# COPY ESSENTIAL CLASSES FROM PART 1
# ═══════════════════════════════════════════════════════════════════════════════

class BreaKHisDataset(Dataset):
    def __init__(self, root_dir: str, split: str = "train", transform=None):
        self.root_dir = Path(root_dir)
        self.split = split
        self.transform = transform
        self.samples = []
        self.labels = []
        self._load_data()
        
    def _load_data(self):
        split_dir = self.root_dir / self.split
        for class_idx, class_name in enumerate(["benign", "malignant"]):
            class_dir = split_dir / class_name
            if class_dir.exists():
                for img_path in sorted(class_dir.glob("*.png")):
                    self.samples.append(str(img_path))
                    self.labels.append(class_idx)
        self.labels = np.array(self.labels)
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img = Image.open(self.samples[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

def get_transforms(is_train: bool = True):
    if is_train:
        return transforms.Compose([
            transforms.Resize((CONFIG.img_size, CONFIG.img_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    return transforms.Compose([
        transforms.Resize((CONFIG.img_size, CONFIG.img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

class SelectiveSSM(nn.Module):
    def __init__(self, d_model: int, d_state: int = 16, expand: int = 2):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_inner = d_model * expand
        self.dt_rank = max(d_model // 16, 1)
        
        self.in_proj = nn.Linear(d_model, self.d_inner * 2, bias=False)
        self.conv1d = nn.Conv1d(self.d_inner, self.d_inner, 4, padding=2, groups=self.d_inner)
        self.x_proj = nn.Linear(self.d_inner, self.dt_rank + d_state * 2, bias=False)
        self.dt_proj = nn.Linear(self.dt_rank, self.d_inner, bias=True)
        
        A = torch.arange(1, d_state + 1, dtype=torch.float32)
        self.A_log = nn.Parameter(torch.log(A))
        self.D = nn.Parameter(torch.ones(self.d_inner))
        
        self.out_proj = nn.Linear(self.d_inner, d_model, bias=False)
        self.norm = nn.LayerNorm(self.d_inner)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, L, _ = x.shape
        xz = self.in_proj(x)
        x_branch, z = xz.chunk(2, dim=-1)
        x_branch = x_branch.transpose(1, 2)
        x_branch = self.conv1d(x_branch)[:, :, :L]
        x_branch = x_branch.transpose(1, 2)
        x_branch = F.silu(x_branch)
        x_dbl = self.x_proj(x_branch)
        dt, B_param, C = torch.split(x_dbl, [self.dt_rank, self.d_state, self.d_state], dim=-1)
        dt = F.softplus(self.dt_proj(dt))
        A = -torch.exp(self.A_log)
        y = self._scan_optimized(x_branch, dt, A, B_param, C)
        y = self.norm(y)
        y = y * F.silu(z)
        return self.out_proj(y)
    
    def _scan_optimized(self, x, dt, A, B, C):
        B_size, L, D = x.shape
        chunk_size = 32
        outputs = []
        h = torch.zeros(B_size, D, self.d_state, device=x.device, dtype=x.dtype)
        for start in range(0, L, chunk_size):
            end = min(start + chunk_size, L)
            chunk_out = []
            for t in range(start, end):
                dt_t = dt[:, t, :].unsqueeze(-1)
                dA = torch.exp(A.view(1, 1, -1) * dt_t)
                dB = dt_t * B[:, t, :].unsqueeze(1)
                h = dA * h + dB * x[:, t, :].unsqueeze(-1)
                y_t = (h * C[:, t, :].unsqueeze(1)).sum(-1) + self.D * x[:, t, :]
                chunk_out.append(y_t)
            outputs.extend(chunk_out)
        return torch.stack(outputs, dim=1)

class MicroMamba(nn.Module):
    def __init__(self, d_model: int, n_layers: int = 2, d_state: int = 16):
        super().__init__()
        self.d_model = d_model
        self.blocks = nn.ModuleList([
            nn.ModuleDict({
                'norm': nn.LayerNorm(d_model),
                'ssm': SelectiveSSM(d_model, d_state),
                'drop': nn.Dropout(0.1)
            }) for _ in range(n_layers)
        ])
        self.agent_embed = nn.Parameter(torch.randn(1, 1, d_model) * 0.02)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B = x.shape[0]
        x = torch.cat([self.agent_embed.expand(B, -1, -1), x], dim=1)
        for block in self.blocks:
            res = x
            x = block['norm'](x)
            x = block['ssm'](x)
            x = block['drop'](x)
            x = res + x
        return x[:, 1:, :]

class MessagePassing(nn.Module):
    def __init__(self, d_model: int, n_agents: int, n_heads: int = 4):
        super().__init__()
        self.n_agents = n_agents
        self.d_model = d_model
        self.d_compress = d_model // 2
        self.compress = nn.Sequential(
            nn.Linear(d_model, self.d_compress),
            nn.GELU(),
            nn.LayerNorm(self.d_compress)
        )
        self.decompress = nn.Linear(self.d_compress, d_model)
        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=0.1, batch_first=True)
        self.alpha = nn.Parameter(torch.tensor(0.3))
        
    def forward(self, states: List[torch.Tensor]) -> List[torch.Tensor]:
        B = states[0].shape[0]
        summaries = torch.stack([self.compress(s.mean(dim=1)) for s in states], dim=1)
        decompressed = self.decompress(summaries)
        attn_out, _ = self.attn(decompressed, decompressed, decompressed)
        alpha = torch.sigmoid(self.alpha)
        updated = []
        for i, s in enumerate(states):
            msg = attn_out[:, i:i+1, :].expand(-1, s.shape[1], -1)
            updated.append(s + alpha * msg)
        return updated

class Consensus(nn.Module):
    def __init__(self, d_model: int, n_agents: int, n_classes: int):
        super().__init__()
        self.n_agents = n_agents
        self.importance = nn.Parameter(torch.ones(n_agents) / n_agents)
        self.agent_heads = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm(d_model),
                nn.Linear(d_model, d_model // 2),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(d_model // 2, n_classes)
            ) for _ in range(n_agents)
        ])
        self.consensus_head = nn.Sequential(
            nn.LayerNorm(d_model * n_agents),
            nn.Linear(d_model * n_agents, d_model),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(d_model, n_classes)
        )
        
    def forward(self, states: List[torch.Tensor]) -> Tuple[torch.Tensor, List[torch.Tensor], Dict]:
        pooled = [s.mean(dim=1) for s in states]
        individual = [head(p) for head, p in zip(self.agent_heads, pooled)]
        concat = torch.cat(pooled, dim=-1)
        consensus_logits = self.consensus_head(concat)
        
        with torch.no_grad():
            preds = torch.stack([torch.argmax(l, dim=-1) for l in individual], dim=1)
            mode_pred = torch.mode(preds, dim=1).values
            agreement = (preds == mode_pred.unsqueeze(1)).float().mean()
            kappas = []
            for i in range(len(individual)):
                for j in range(i+1, len(individual)):
                    p_i = torch.argmax(individual[i], dim=-1).cpu().numpy()
                    p_j = torch.argmax(individual[j], dim=-1).cpu().numpy()
                    if len(np.unique(p_i)) > 1 and len(np.unique(p_j)) > 1:
                        try:
                            kappas.append(cohen_kappa_score(p_i, p_j))
                        except:
                            pass
        
        metrics = {
            'consensus_rate': agreement.item(),
            'mean_kappa': np.mean(kappas) if kappas else 1.0
        }
        return consensus_logits, individual, metrics

class SwarmMamba(nn.Module):
    def __init__(self, img_size: int = 224, patch_size: int = 16, in_chans: int = 3,
                 n_agents: int = 4, d_model: int = 80, n_layers: int = 2,
                 n_classes: int = 2, n_comm_rounds: int = 2, d_state: int = 16,
                 use_communication: bool = True):
        super().__init__()
        self.n_agents = n_agents
        self.n_comm_rounds = n_comm_rounds
        self.use_communication = use_communication
        self.n_patches = (img_size // patch_size) ** 2
        self.patch_embed = nn.Conv2d(in_chans, d_model, patch_size, patch_size)
        self.pos_embed = nn.Parameter(torch.randn(1, self.n_patches, d_model) * 0.02)
        self.agents = nn.ModuleList([
            MicroMamba(d_model, n_layers, d_state) for _ in range(n_agents)
        ])
        self.message_passing = MessagePassing(d_model, n_agents)
        self.consensus = Consensus(d_model, n_agents, n_classes)
        
    def partition(self, x: torch.Tensor) -> List[torch.Tensor]:
        B, N, D = x.shape
        per_agent = N // self.n_agents
        return [x[:, i*per_agent:(i+1)*per_agent if i < self.n_agents-1 else N, :].clone() 
                for i in range(self.n_agents)]
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, List[torch.Tensor], Dict]:
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        x = x + self.pos_embed
        partitions = self.partition(x)
        states = [agent(part) for agent, part in zip(self.agents, partitions)]
        for _ in range(self.n_comm_rounds):
            if self.use_communication:
                states = self.message_passing(states)
            states = [agent(s) for agent, s in zip(self.agents, states)]
        return self.consensus(states)

class VisionMamba(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, d_model=82, 
                 n_layers=9, n_classes=2, d_state=16):
        super().__init__()
        self.n_patches = (img_size // patch_size) ** 2
        self.patch_embed = nn.Conv2d(in_chans, d_model, patch_size, patch_size)
        self.pos_embed = nn.Parameter(torch.randn(1, self.n_patches, d_model) * 0.02)
        self.blocks = nn.ModuleList([
            nn.ModuleDict({
                'norm': nn.LayerNorm(d_model),
                'ssm': SelectiveSSM(d_model, d_state)
            }) for _ in range(n_layers)
        ])
        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(d_model // 2, n_classes)
        )
        
    def forward(self, x):
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        x = x + self.pos_embed
        for block in self.blocks:
            x = x + block['ssm'](block['norm'](x))
        return self.head(x.mean(dim=1))

class Timer:
    def __init__(self):
        self.times = {}
        
    def start(self, name):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        self.times[name] = {'start': time.perf_counter()}
        
    def stop(self, name):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        self.times[name]['end'] = time.perf_counter()
        self.times[name]['duration'] = self.times[name]['end'] - self.times[name]['start']
        return self.times[name]['duration']

def train_epoch(model, loader, optimizer, criterion, device, is_swarm=False):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        if is_swarm:
            logits, individual, _ = model(batch_x)
            loss = criterion(logits, batch_y)
            for ind in individual:
                loss += 0.1 * criterion(ind, batch_y)
        else:
            logits = model(batch_x)
            loss = criterion(logits, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item() * batch_x.size(0)
        preds = torch.argmax(logits, dim=-1)
        correct += (preds == batch_y).sum().item()
        total += batch_y.size(0)
    return total_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader, device, is_swarm=False):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    for batch_x, batch_y in loader:
        batch_x, batch_y = batch_x.to(device, non_blocking=True), batch_y.to(device, non_blocking=True)
        if is_swarm:
            logits, _, _ = model(batch_x)
        else:
            logits = model(batch_x)
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    return {
        'accuracy': accuracy_score(all_labels, all_preds),
        'auc': roc_auc_score(all_labels, all_probs) if len(np.unique(all_labels)) > 1 else 0.5,
        'f1': f1_score(all_labels, all_preds, average='weighted')
    }

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION T3: NOISE ROBUSTNESS TESTING
# ═══════════════════════════════════════════════════════════════════════════════

class NoisyTransform:
    """Apply Gaussian noise to images"""
    def __init__(self, noise_std: float = 0.1):
        self.noise_std = noise_std
        
    def __call__(self, tensor):
        noise = torch.randn_like(tensor) * self.noise_std
        return torch.clamp(tensor + noise, 0, 1)

def run_T3_noise_robustness():
    print("\n" + "═" * 70)
    print("EXPERIMENT T3: NOISE ROBUSTNESS TESTING")
    print("═" * 70)
    
    timer = Timer()
    timer.start('T3_total')
    
    noise_levels = [0.0, 0.05, 0.10, 0.15, 0.20]
    
    results = {
        'Swarm-Mamba': {level: [] for level in noise_levels},
        'Vision-Mamba': {level: [] for level in noise_levels}
    }
    
    test_ds = BreaKHisDataset(CONFIG.data_root, "test", get_transforms(False))
    
    for seed in CONFIG.seeds:
        print(f"\n{'─' * 50}")
        print(f"Seed: {seed}")
        print('─' * 50)
        set_seed(seed)
        clear_memory()
        
        train_ds = BreaKHisDataset(CONFIG.data_root, "train", get_transforms(True))
        train_loader = DataLoader(train_ds, batch_size=CONFIG.batch_size, shuffle=True,
                                 num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory)
        
        for model_name in ['Swarm-Mamba', 'Vision-Mamba']:
            print(f"\n  Training {model_name}...")
            
            if model_name == 'Swarm-Mamba':
                model = SwarmMamba(
                    img_size=CONFIG.img_size, patch_size=CONFIG.patch_size,
                    n_agents=CONFIG.n_agents, d_model=CONFIG.d_model,
                    n_layers=CONFIG.n_layers, n_classes=CONFIG.n_classes,
                    n_comm_rounds=CONFIG.n_comm_rounds, d_state=CONFIG.d_state
                ).to(CONFIG.device)
                is_swarm = True
            else:
                model = VisionMamba(
                    img_size=CONFIG.img_size, patch_size=CONFIG.patch_size,
                    d_model=82, n_layers=9, n_classes=CONFIG.n_classes, d_state=CONFIG.d_state
                ).to(CONFIG.device)
                is_swarm = False
            
            optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG.lr, weight_decay=CONFIG.weight_decay)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CONFIG.epochs)
            criterion = nn.CrossEntropyLoss()
            
            for epoch in range(CONFIG.epochs):
                train_epoch(model, train_loader, optimizer, criterion, CONFIG.device, is_swarm)
                scheduler.step()
            
            for noise_level in noise_levels:
                if noise_level == 0:
                    test_transform = get_transforms(False)
                else:
                    test_transform = transforms.Compose([
                        transforms.Resize((CONFIG.img_size, CONFIG.img_size)),
                        transforms.ToTensor(),
                        NoisyTransform(noise_level),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ])
                
                noisy_test_ds = BreaKHisDataset(CONFIG.data_root, "test", test_transform)
                test_loader = DataLoader(noisy_test_ds, batch_size=CONFIG.batch_size, shuffle=False,
                                         num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory)
                
                result = evaluate(model, test_loader, CONFIG.device, is_swarm)
                results[model_name][noise_level].append(result['accuracy'])
                
                print(f"    Noise σ={noise_level:.2f}: Acc={result['accuracy']:.4f}")
            
            del model
            clear_memory()
    
    t3_time = timer.stop('T3_total')
    
    print("\n" + "═" * 70)
    print("T3 RESULTS SUMMARY")
    print("═" * 70)
    
    print(f"\n{'Model':<15} {'Clean':>12} {'σ=0.05':>12} {'σ=0.10':>12} {'σ=0.15':>12} {'σ=0.20':>12} {'Degradation':>12}")
    print("─" * 90)
    
    for model_name in ['Swarm-Mamba', 'Vision-Mamba']:
        clean_acc = np.mean(results[model_name][0.0])
        worst_acc = np.mean(results[model_name][0.20])
        degradation = (clean_acc - worst_acc) / clean_acc * 100
        
        row = f"{model_name:<15}"
        for noise in noise_levels:
            acc_mean = np.mean(results[model_name][noise])
            acc_std = np.std(results[model_name][noise])
            row += f" {acc_mean:.3f}±{acc_std:.3f}"
        row += f"  {degradation:>10.1f}%"
        print(row)
    
    print("\n" + "─" * 70)
    print("ROBUSTNESS COMPARISON")
    print("─" * 70)
    
    swarm_deg = (np.mean(results['Swarm-Mamba'][0.0]) - np.mean(results['Swarm-Mamba'][0.20])) / np.mean(results['Swarm-Mamba'][0.0]) * 100
    vim_deg = (np.mean(results['Vision-Mamba'][0.0]) - np.mean(results['Vision-Mamba'][0.20])) / np.mean(results['Vision-Mamba'][0.0]) * 100
    
    print(f"Swarm-Mamba Degradation: {swarm_deg:.2f}%")
    print(f"Vision-Mamba Degradation: {vim_deg:.2f}%")
    
    if swarm_deg < vim_deg:
        improvement = vim_deg - swarm_deg
        print(f"✓ Swarm-Mamba is {improvement:.2f}% more robust")
    else:
        print(f"✗ Vision-Mamba is more robust")
    
    print(f"\nT3 Total Execution Time: {t3_time/60:.2f} minutes")
    
    return results

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION T4: COMMUNICATION ABLATION
# ═══════════════════════════════════════════════════════════════════════════════

def run_T4_communication_ablation():
    print("\n" + "═" * 70)
    print("EXPERIMENT T4: COMMUNICATION ABLATION STUDY")
    print("═" * 70)
    
    timer = Timer()
    timer.start('T4_total')
    
    results = {
        'With Communication': [],
        'Without Communication': []
    }
    
    for seed in CONFIG.seeds:
        print(f"\n{'─' * 50}")
        print(f"Seed: {seed}")
        print('─' * 50)
        set_seed(seed)
        clear_memory()
        
        train_ds = BreaKHisDataset(CONFIG.data_root, "train", get_transforms(True))
        test_ds = BreaKHisDataset(CONFIG.data_root, "test", get_transforms(False))
        
        train_loader = DataLoader(train_ds, batch_size=CONFIG.batch_size, shuffle=True,
                                 num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory)
        test_loader = DataLoader(test_ds, batch_size=CONFIG.batch_size, shuffle=False,
                                num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory)
        
        for use_comm, name in [(True, 'With Communication'), (False, 'Without Communication')]:
            print(f"\n  Training Swarm-Mamba {name}...")
            
            model = SwarmMamba(
                img_size=CONFIG.img_size, patch_size=CONFIG.patch_size,
                n_agents=CONFIG.n_agents, d_model=CONFIG.d_model,
                n_layers=CONFIG.n_layers, n_classes=CONFIG.n_classes,
                n_comm_rounds=CONFIG.n_comm_rounds, d_state=CONFIG.d_state,
                use_communication=use_comm
            ).to(CONFIG.device)
            
            optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG.lr, weight_decay=CONFIG.weight_decay)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CONFIG.epochs)
            criterion = nn.CrossEntropyLoss()
            
            for epoch in range(CONFIG.epochs):
                train_epoch(model, train_loader, optimizer, criterion, CONFIG.device, is_swarm=True)
                scheduler.step()
            
            result = evaluate(model, test_loader, CONFIG.device, is_swarm=True)
            results[name].append(result['accuracy'])
            
            print(f"    Accuracy: {result['accuracy']:.4f}")
            
            del model
            clear_memory()
    
    t4_time = timer.stop('T4_total')
    
    print("\n" + "═" * 70)
    print("T4 RESULTS SUMMARY")
    print("═" * 70)
    
    with_comm = np.array(results['With Communication'])
    without_comm = np.array(results['Without Communication'])
    
    print(f"\nWith Communication:    {np.mean(with_comm):.4f} ± {np.std(with_comm):.4f}")
    print(f"Without Communication: {np.mean(without_comm):.4f} ± {np.std(without_comm):.4f}")
    
    t_stat, p_val = stats.ttest_rel(with_comm, without_comm)
    improvement = (np.mean(with_comm) - np.mean(without_comm)) / np.mean(without_comm) * 100
    
    print(f"\nPaired t-test:")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_val:.4f}")
    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
    print(f"  Significance: {sig if sig else 'Not significant'}")
    print(f"  Improvement: {improvement:.2f}%")
    
    if p_val < 0.05:
        print(f"\n✓ Communication provides statistically significant improvement")
    else:
        print(f"\n✗ No significant difference found")
    
    print(f"\nT4 Total Execution Time: {t4_time/60:.2f} minutes")
    
    return results

# ═══════════════════════════════════════════════════════════════════════════════
# SECTION T5: DETAILED INFERENCE LATENCY
# ═══════════════════════════════════════════════════════════════════════════════

@torch.no_grad()
def detailed_inference_benchmark(model, loader, device, is_swarm=False):
    model.eval()
    batch_x, _ = next(iter(loader))
    batch_x = batch_x.to(device)
    
    # Warmup
    for _ in range(10):
        if is_swarm:
            _ = model(batch_x)
        else:
            _ = model(batch_x)
    
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    
    # Measure
    times = []
    for _ in range(100):
        start = time.perf_counter()
        if is_swarm:
            _ = model(batch_x)
        else:
            _ = model(batch_x)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        times.append(time.perf_counter() - start)
    
    times = np.array(times) * 1000  # Convert to ms
    
    return {
        'mean': np.mean(times),
        'std': np.std(times),
        'min': np.min(times),
        'max': np.max(times),
        'median': np.median(times),
        'p95': np.percentile(times, 95),
        'p99': np.percentile(times, 99),
        'per_sample': np.mean(times) / batch_x.size(0),
        'throughput': batch_x.size(0) / (np.mean(times) / 1000)
    }

def run_T5_latency_benchmarking():
    print("\n" + "═" * 70)
    print("EXPERIMENT T5: DETAILED INFERENCE LATENCY BENCHMARKING")
    print("═" * 70)
    
    timer = Timer()
    timer.start('T5_total')
    
    set_seed(42)
    
    test_ds = BreaKHisDataset(CONFIG.data_root, "test", get_transforms(False))
    test_loader = DataLoader(test_ds, batch_size=CONFIG.batch_size, shuffle=False,
                            num_workers=CONFIG.num_workers, pin_memory=CONFIG.pin_memory)
    
    models = {
        'Swarm-Mamba': SwarmMamba(
            img_size=CONFIG.img_size, patch_size=CONFIG.patch_size,
            n_agents=CONFIG.n_agents, d_model=CONFIG.d_model,
            n_layers=CONFIG.n_layers, n_classes=CONFIG.n_classes,
            n_comm_rounds=CONFIG.n_comm_rounds, d_state=CONFIG.d_state
        ),
        'Vision-Mamba': VisionMamba(
            img_size=CONFIG.img_size, patch_size=CONFIG.patch_size,
            d_model=82, n_layers=9, n_classes=CONFIG.n_classes, d_state=CONFIG.d_state
        )
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nBenchmarking {name}...")
        model = model.to(CONFIG.device)
        is_swarm = 'Swarm' in name
        
        bench = detailed_inference_benchmark(model, test_loader, CONFIG.device, is_swarm)
        results[name] = bench
        
        print(f"  Batch Time (ms):  {bench['mean']:.2f} ± {bench['std']:.2f}")
        print(f"  Per-Sample (ms):  {bench['per_sample']:.3f}")
        print(f"  Throughput:       {bench['throughput']:.1f} samples/s")
        print(f"  Median:           {bench['median']:.2f} ms")
        print(f"  95th percentile:  {bench['p95']:.2f} ms")
        print(f"  99th percentile:  {bench['p99']:.2f} ms")
        
        del model
        clear_memory()
    
    t5_time = timer.stop('T5_total')
    
    print("\n" + "═" * 70)
    print("T5 SUMMARY TABLE")
    print("═" * 70)
    
    print(f"\n{'Metric':<20} {'Swarm-Mamba':>15} {'Vision-Mamba':>15} {'Speedup':>10}")
    print("─" * 65)
    
    metrics = [
        ('Batch Time (ms)', 'mean'),
        ('Per-Sample (ms)', 'per_sample'),
        ('Throughput (samp/s)', 'throughput'),
        ('P95 Latency (ms)', 'p95'),
        ('P99 Latency (ms)', 'p99')
    ]
    
    for label, key in metrics:
        swarm_val = results['Swarm-Mamba'][key]
        vim_val = results['Vision-Mamba'][key]
        if 'throughput' in key.lower():
            speedup = swarm_val / vim_val
        else:
            speedup = vim_val / swarm_val
        print(f"{label:<20} {swarm_val:>15.2f} {vim_val:>15.2f} {speedup:>9.2f}x")
    
    print(f"\nT5 Total Execution Time: {t5_time/60:.2f} minutes")
    
    return results

# ═══════════════════════════════════════════════════════════════════════════════
# FINAL PUBLICATION TABLES
# ═══════════════════════════════════════════════════════════════════════════════

def generate_publication_tables(t3_results, t4_results, t5_results):
    print("\n" + "═" * 70)
    print("PUBLICATION-READY SUMMARY TABLES")
    print("═" * 70)
    
    # TABLE 1: Main Comparison (from Part 1 results - hardcoded from output)
    print("\n" + "─" * 70)
    print("TABLE 1: Comparative Performance (Parameter-Matched ~520K)")
    print("─" * 70)
    
    print(f"\n{'Model':<15} {'Params':>10} {'AUC':>15} {'Accuracy':>15} {'F1':>15}")
    print("─" * 70)
    print(f"{'Swarm-Mamba':<15} {'524,343':>10} {'0.8128±0.0061':>15} {'0.8391±0.0046':>15} {'0.8300±0.0063':>15}")
    print(f"{'Vision-Mamba':<15} {'522,773':>10} {'0.8030±0.0018':>15} {'0.8483±0.0048':>15} {'0.8419±0.0054':>15}")
    print(f"{'ViT':<15} {'520,814':>10} {'0.7832±0.0009':>15} {'0.8251±0.0090':>15} {'0.8193±0.0085':>15}")
    
    # TABLE 2: Consensus Analysis
    print("\n" + "─" * 70)
    print("TABLE 2: Agent Consensus Analysis")
    print("─" * 70)
    print("\nMetric                          Value")
    print("─" * 45)
    print(f"Overall Consensus Rate          98.18% ± 0.38%")
    print(f"Inter-Agent Kappa               0.8238 ± 0.0725")
    print(f"Consensus on Correct            98.61% ± 0.31%")
    print(f"Consensus on Incorrect          95.27% ± 0.80%")
    print(f"Agent Diversity (σ)             0.0022")
    
    # TABLE 3: Noise Robustness
    print("\n" + "─" * 70)
    print("TABLE 3: Noise Robustness Comparison")
    print("─" * 70)
    
    print(f"\n{'Model':<15} {'Clean':>10} {'σ=0.05':>10} {'σ=0.10':>10} {'σ=0.15':>10} {'σ=0.20':>10} {'Deg.':>8}")
    print("─" * 75)
    
    for model_name in ['Swarm-Mamba', 'Vision-Mamba']:
        clean = np.mean(t3_results[model_name][0.0])
        worst = np.mean(t3_results[model_name][0.20])
        deg = (clean - worst) / clean * 100
        
        row = f"{model_name:<15}"
        for noise in [0.0, 0.05, 0.10, 0.15, 0.20]:
            acc = np.mean(t3_results[model_name][noise])
            row += f" {acc:>9.3f}"
        row += f" {deg:>7.1f}%"
        print(row)
    
    # TABLE 4: Communication Ablation
    print("\n" + "─" * 70)
    print("TABLE 4: Communication Ablation")
    print("─" * 70)
    
    with_comm = np.array(t4_results['With Communication'])
    without_comm = np.array(t4_results['Without Communication'])
    improvement = (np.mean(with_comm) - np.mean(without_comm)) / np.mean(without_comm) * 100
    t_stat, p_val = stats.ttest_rel(with_comm, without_comm)
    
    print(f"\n{'Configuration':<25} {'Accuracy':>15} {'Δ vs No-Comm':>15} {'p-value':>10}")
    print("─" * 70)
    print(f"{'With Communication':<25} {np.mean(with_comm):.4f}±{np.std(with_comm):.4f}  {'+'+str(round(improvement, 2))+'%':>15} {p_val:>10.4f}")
    print(f"{'Without Communication':<25} {np.mean(without_comm):.4f}±{np.std(without_comm):.4f}  {'baseline':>15} {'-':>10}")
    
    # TABLE 5: Latency Comparison
    print("\n" + "─" * 70)
    print("TABLE 5: Inference Latency Analysis")
    print("─" * 70)
    
    print(f"\n{'Metric':<25} {'Swarm-Mamba':>15} {'Vision-Mamba':>15} {'Relative':>10}")
    print("─" * 70)
    
    swarm_time = t5_results['Swarm-Mamba']['per_sample']
    vim_time = t5_results['Vision-Mamba']['per_sample']
    swarm_tp = t5_results['Swarm-Mamba']['throughput']
    vim_tp = t5_results['Vision-Mamba']['throughput']
    
    print(f"{'Per-Sample Latency (ms)':<25} {swarm_time:>15.3f} {vim_time:>15.3f} {vim_time/swarm_time:>9.2f}x")
    print(f"{'Throughput (samples/s)':<25} {swarm_tp:>15.1f} {vim_tp:>15.1f} {swarm_tp/vim_tp:>9.2f}x")
    print(f"{'P95 Latency (ms)':<25} {t5_results['Swarm-Mamba']['p95']:>15.2f} {t5_results['Vision-Mamba']['p95']:>15.2f} {t5_results['Vision-Mamba']['p95']/t5_results['Swarm-Mamba']['p95']:>9.2f}x")

# ═══════════════════════════════════════════════════════════════════════════════
# MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    print("\n" + "█" * 70)
    print("█" + " " * 68 + "█")
    print("█" + "   SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS".center(68) + "█")
    print("█" + "   PART 2: Advanced Experiments & Analysis".center(68) + "█")
    print("█" + " " * 68 + "█")
    print("█" * 70)
    
    total_timer = Timer()
    total_timer.start('total')
    
    # Check data
    data_path = Path(CONFIG.data_root)
    if not data_path.exists():
        print(f"\nERROR: Data not found at {CONFIG.data_root}")
        sys.exit(1)
    
    # Run experiments
    print("\n" + "█" * 70)
    t3_results = run_T3_noise_robustness()
    
    print("\n" + "█" * 70)
    t4_results = run_T4_communication_ablation()
    
    print("\n" + "█" * 70)
    t5_results = run_T5_latency_benchmarking()
    
    # Generate publication tables
    print("\n" + "█" * 70)
    generate_publication_tables(t3_results, t4_results, t5_results)
    
    # Final summary
    total_time = total_timer.stop('total')
    final_mem = get_memory_stats()
    
    print("\n" + "═" * 70)
    print("PART 2 EXECUTION COMPLETE")
    print("═" * 70)
    print(f"\nTotal Execution Time: {total_time/60:.2f} minutes ({total_time:.1f} seconds)")
    print(f"Peak GPU Memory Used: {final_mem['max']:.2f} GB")
    
    print("\n" + "─" * 70)
    print("ALL EXPERIMENTS COMPLETED")
    print("─" * 70)
    print("  ✓ T1: Comparative Benchmarking (Part 1)")
    print("  ✓ T2: Consensus Analysis (Part 1)")
    print("  ✓ T3: Noise Robustness Testing")
    print("  ✓ T4: Communication Ablation Study")
    print("  ✓ T5: Inference Latency Benchmarking")
    print("  ✓ Publication-Ready Tables Generated")
    print("═" * 70)

══════════════════════════════════════════════════════════════════════
SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS - PART 2
══════════════════════════════════════════════════════════════════════
Device: cuda
GPU: Tesla P100-PCIE-16GB
══════════════════════════════════════════════════════════════════════

██████████████████████████████████████████████████████████████████████
█                                                                    █
█                SWARM-MAMBA: Q1 PUBLICATION EXPERIMENTS             █
█                PART 2: Advanced Experiments & Analysis             █
█                                                                    █
██████████████████████████████████████████████████████████████████████

██████████████████████████████████████████████████████████████████████

══════════════════════════════════════════════════════════════════════
EXPERIMENT T3: NOISE ROBUSTNESS TESTING
══════════════════════════════════════════════════════════════════════

────────────────